This is the proposed pipeline of the paper **Large Language Models for Natural Language Generation for Declarative Processes**. It contains the packages, utilities, prompts used for generation and the synthetic data generator using OpenAI API.

#Requirements

##Install dependencies

In [ ]:
!pip install declare4py
!pip install openai
!pip install pm4py
!pip install pyairports
!pip install pycountry
!pip install pydantic
!pip install pydantic_core
!pip install requests
!pip install requests-oauthlib
!pip install scipy
!pip install tenacity
!pip install termcolor
!pip install tiktoken
!pip install crewai crewai-tools
!pip install lapjv
!pip install -U sentence-transformers
!pip install spacy textstat pandas markdown bs4
!pip install spacy-readability

##Import dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import auth
from google.auth import default
import gspread
import pm4py
import Declare4Py as dec4py
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
import requests
from google.colab import userdata
from openai import OpenAI
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from enum import Enum
import os
from pydantic import BaseModel, Field, conlist, conint, ValidationError, field_validator, model_validator
import re
import logging
import crewai
from crewai import Agent, Task, Process, Crew, LLM
from typing import List, Optional, Dict, Union, Literal
import zipfile
import spacy
from markdown import markdown
from bs4 import BeautifulSoup
import spacy.cli
spacy.cli.download("en_core_web_sm")
from spacy_readability import Readability
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from lapjv import lapjv
import copy
import networkx as nx
from collections import Counter
import math

##Utilities

This section is dedidated to set utilities to interplay with OpenAI API and set environment variables and model. It's necessary to have access to OpenAI key and have funds. You can set your OpenAI API key [here](https://platform.openai.com/settings/profile?tab=api-keys).

In [ ]:
# OpenAI environment variables configuration
GPT_MODEL = "gpt-4o-2024-11-20"
JUDGE_MODEL="o1-preview-2024-09-12"
seed = 42
client = OpenAI(api_key=userdata.get('openai_key'))
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')
os.environ["OPENAI_MODEL_NAME"] = GPT_MODEL
embedding_model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, temperature, seed=seed,function_call=None, tools=None, tool_choice=None, model=GPT_MODEL):
    """
    Send a chat completion request to the GPT model and handle retries.

    This function sends a request to generate a chat completion based on the input messages and
    parameters. It automatically retries up to three times with exponential backoff in case of failure.

    Args:
        messages (list): A list of message dictionaries representing the conversation history.
        temperature (float): Sampling temperature to use. Higher values will make the output more random.
        function_call (dict, optional): A dictionary specifying a function call request. Defaults to None.
        seed (int): A random seed value. Defaults to 42.
        model (str): The GPT model to use for generating the completion. Defaults to `GPT_MODEL`.
        function_call (dict, optional
        tools (list, optional): A list of tools that the assistant can use. Defaults to None.
        tool_choice (str, optional): A specific tool choice to be used by the assistant. Defaults to None.
        model (str, optional): The GPT model to use for generating the completion. Defaults to `GPT_MODEL`.

    Returns:
        dict or Exception: Returns the response from the GPT model if successful, otherwise returns the exception.

    Raises:
        Exception: If unable to generate a ChatCompletion response after retries.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=messages,
            seed=seed,
            tools=tools,
            function_call=function_call,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def structured_output(messages, temperature=None, seed=seed, response_format=None,model=GPT_MODEL):
    try:
        response = client.beta.chat.completions.parse(
            model=model,
            temperature=temperature,
            seed=seed,
            messages=messages,
            response_format=response_format,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def pretty_print_conversation(messages):
    """
    Print the conversation in a human-readable format with role-based color coding.

    This function takes a list of messages and prints each message with color coding based
    on the role of the sender (e.g., system, user, assistant, function).

    Args:
        messages (list): A list of message dictionaries representing the conversation history.

    Example:
        pretty_print_conversation([{'role': 'user', 'content': 'Hello!'}])

    """
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

# Extend class DeclareModel to add new methods to return attributes and binds.

# Configure the logger
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True,
    handlers=[
        logging.StreamHandler()
    ]
)

# Create a logger instance
logger = logging.getLogger('MyLogger')

class ExtendedDeclareModel(DeclareModel):
    """
    Extended version of the DeclareModel class from declare4py.

    This class adds two new methods to retrieve attributes and binds.
    """

    def get_decl_attributes(self):
        """
        Retrieve the list of attributes.

        This method iterates through the attributes list of the parsed model,
        collects attribute names and their values (if present), and returns a
        formatted list of attributes.

        Returns:
            list: A list of strings representing the attributes.
        """
        attributes = []
        for attr_name, attr_obj in self.parsed_model.attributes_list.items():
            attr_value = ""
            if attr_obj.attr_value:
                attr_value = ": " + attr_obj.attr_value.value_original
            attributes.append(f"{attr_name}{attr_value}")
        return attributes

    def get_decl_binds(self):
        """
        Retrieve the list of binds.

        This method iterates through the events in the parsed model, collects
        event names and their bound attributes, and returns a formatted list of
        binds.

        Returns:
            list: A list of strings representing the  binds.
        """
        binds = []
        for event_type, events in self.parsed_model.events.items():
            for event_name, event_obj in events.items():
                if event_obj.attributes:
                    bound_attrs = ", ".join(attr_obj.get_name() for attr_obj in event_obj.attributes.values())
                    binds.append(f"bind {event_name}: {bound_attrs}")
        return binds


###Prompts and Tools

In [ ]:
response_template="""
The MP-Declare process model for the property transaction process is described as follows:

### **Trace-Related Constraints**
#### **Init Constraints**
The process begins with the activity 'Draft Contract,' which involves creating the initial version of the contract with 'Contract Version' and 'Contract Status' attributes.
#### **End Constraints**
The process concludes with the activity 'Close Deal,' which involves finalizing the property transaction and completing all necessary paperwork. This activity must be the last to occur in the process and must be completed within 30 days.

### **Frequency Constraints**

#### **Existence Constraint**
The 'Review Contract' activity must occur at least once in the process. This ensures that the contract is examined for accuracy and completeness, adding depth and reliability to the process.

### **Flexible Ordering Constraints**

#### **Precedence Constraints**
1. The activity 'Approve Contract' is permitted to occur only if a 'Conduct Title Search' activity has occurred before it. The 'Conduct Title Search' activity must verify that the 'Contract Status is Reviewed' and the 'Title Search Result is Clear,' ensuring that the contract is thoroughly reviewed and the property ownership is legally verified before approval.
2. The activity 'Negotiate Terms' is permitted to occur only if an 'Assess Property Value' activity has occurred before it, with the condition that the 'Property Value' is greater than 50,000. This ensures that the negotiation of terms is based on a valid and substantial property valuation.
3. The activity 'Close Deal' is permitted to occur only if a 'Verify Funds' activity has occurred before it. The 'Verify Funds' activity must confirm that the 'Deal Status is Open' and that the 'Funds Verified' are greater than or equal to the 'Property Value,' ensuring financial readiness before finalizing the deal.

#### **Response Constraints**
1. If the 'Draft Contract' activity occurs with the condition 'Contract Status is Draft,' then the 'Review Contract' activity must occur within 2 days. This ensures a timely review of the drafted contract, maintaining the process's momentum.
2. If the 'Sign Contract' activity occurs with the condition 'Contract Status is Signed,' then the 'Register Property' activity must occur with the condition 'Registration Status is Pending' within 5 days. This ensures that the property registration follows promptly after the contract is signed, maintaining the legal and procedural flow.

#### **Succession Constraints**
Whenever an 'Approve Contract' activity occurs with the condition 'Contract Status is Approved,' it must be immediately followed by a 'Sign Contract' activity with the condition 'Contract Status == Contract Status' within 1 day. This establishes a direct causal link between the approval and signing of the contract, ensuring a seamless progression in the process.

### **Exclusivity Constraints**

#### **Not Co-Existence Constraints**
The 'Sign Contract' activity and the 'Conduct Title Search' activity cannot both occur within the same process instance if the conditions 'Contract Status is Approved' and 'Title Search Result is Issues Found' are met. This exclusion ensures that a contract cannot be signed if the title search reveals issues, thereby maintaining the integrity of the process by preventing conflicting events.

### **Immediate Ordering Constraints**

#### **Chain Succession Constraints**
Whenever an 'Approve Contract' activity occurs with the condition 'Contract Status is Approved,' it must be immediately followed by a 'Sign Contract' activity with the condition 'Contract Status == Contract Status' within 1 day. This tightly coupled sequence ensures that the approval and signing of the contract occur in immediate succession, leaving no room for delays or intervening activities.
"""

# The assistant prompt used to generate the intermediary descriptions with the rethorical relations.
decltotextassistant="""
<Assistant Prompt>
You are an an expert in declarative business process management, specifically for interpreting and describing Multi-Perspective Declare models (MP-Declare). Your primary task is to analyze the provided MP-Declare model and generate a comprehensive and consistent description. This description should meticulously reflect the structure and terminology of the model, maintaining exact naming and case sensitivity for model parts (activities, attributes, binds, and constraints).
Use the following knowledge base to know how to interpret the MP-Declare model:
<MP-Declare Model Specification>
#**Multi-Perspective Declare (MP-Declare) Constructs**

##**Activities**

<Activities Specification Guidelines>
Activities are specified with the word 'activity' followed by descriptive names, which should use active verbs, avoid abbreviations, and follow naming conventions (i.e., <Action Verb> <Object> or <Object> <Action Verb>).
</Activities Specification Guidelines>

**Examples of activities specification:**
<Activities Examples>
activity Driving Test
activity Initiate Purchase Order
</Activities Examples>
</Activities Specification Guidelines>

##**Binds**

<Binds Specification Guidelines>
Binds are specified as a link between activities and attributes. They have as a prefix the activity name followed by ':' and then a list of attributes, each separated by ','.

**Examples of binds specification:**
<Binds Examples>
Driving Test: Driver, Grade
Initiate Purchase Order: Purchase ID, Customer ID, Order Status, Order Value
</Binds Examples>
</Binds Specification Guidelines>

##**Attributes**
<Attributes Specification Guidelines>
MP-Declare models supports three types of attributes: integer, float and enumeration.

**Attribute Types:**
- **Integer:** Represents numeric integer values within a specified range.
- **Float:** Represents numeric values with floating-point within a specified range.
- **Enumeration:** Represents a limited set of possible values.

**Integer specification:**<Integer Attribute Specification Guidelines>[{Attribute Name}: integer between {min_value} and {max_value}], where {attribute name} is the name of attribute and {min_value} and {max_value} are two integer numbers.</Integer Attribute Specification Guideline>
**Float specification:**<Float Attribute Specification Guideline>[{Attribute Name}: float between {min_value} and {max_value}], where {attribute name} is the name of attribute and {min_value} and {max_value} are two float numbers.</Float Attribute Specification Guideline>
**Enumeration specification:**<Enumeration Specification Guideline>[{Attribute Name}: {value1}, {value2}, ..., {valuen}], where {attribute name} is the name of attribute and {value1} to {valuen} represent possible values for the attribute. These possible values should always be at least 2 and not infinite.</Enumeration Specification Guidelines>

**Examples of attributes specification:**
<Attributes Examples>
**Integer attribute example:** <Integer Attribute Example>Grade: integer between 1 and 5</Integer Attribute Example>.
**Float attribute example:** <Float Attribute Example>Order Value: float between 1.99 and 99999.9</Float Attribute Example>.
**Enumeration attribute example:** <Enumeration Attribute Example>Order Status: Approved, Not Approved, Shipped, Not Shipped</Enumeration Attribute Example>.
</Attributes Examples>
</Attributes Specification Guidelines>

##**Constraints**

MP-Declare has two types of templates: unary and binary. Following is specified the templates and their semantics.

###**Unary Templates**

Unary templates have only Activation activity.

<Unary Constraints Templates>
Constraint Template: Absence[n][Activation]
Semantics: Activation must not occur n times in the process.

Constraint Template: Exactly[n][Activation]
Semantics: Activation must occur exactly n times in the process.

Constraint Template: Existence[n][Activation]
Semantics: Activation must occur at least n times in the process.

Constraint Template: Init[Activation]
Semantics: Activation must be the first activity to occur in the process.

Constraint Template: End[Activation]
Semantics: Activation must be the last activity to occur in the process.
</Unary Constraints Templates>

###**Binary Templates**
Binary templates have both Activation and Target activities.
####**Binary Positive Templates**

<Binary Positive Templates>
Constraint Template: Alternate Precedence[Target, Activation]
Semantics: An Activation event can occur only if it was immediately preceded by a Target event, and no other Target event is allowed between that Target and the Activation. This ensures a one-to-one and ordered pairing between Targets and Activations, preventing any Activation from occurring without the necessary prior Target and avoiding overlapping or multiple Targets before an Activation.

Constraint Template: Alternate Response[Activation, Target]
Semantics: If an Activation occurs, then a Target must occur after that Activation and before any subsequent Activation occurs no other occurrence of Activation is allowed between an Activation and its corresponding Target. This ensures that each Activation is individually responded to by a Target in sequence.

Constraint Template: Alternate Succession[Activation, Target]
Semantics: Each occurrence of an Activation must eventually be followed by a Target, and each occurrence of a Target must be preceded by an Activation, with no intermediate occurrences of Activation or Target between them. This ensures a strict one-to-one and ordered pairing between Activations and Targets, preventing any overlapping or interruption by additional Activations or Targets in the sequence.

Constraint Template: Chain Precedence[Target, Activation]
Semantics: A Target event can occur only if it is immediately preceded by an Activation event, with no other events occurring in between. There can be multiple Activation-Target pairs within the process, each adhering to this immediate precedence rule. Activations may occur without being followed by a Target, but Targets cannot occur without such an immediate preceding Activation.

Constraint Template: Chain Response[Activation, Target]
Semantics: Whenever an Activation occurs, it must be immediately followed by a Target in the next event position, with no other events in between. This immediate succession ensures a direct and uninterrupted transition from Activation to Target. Multiple Activation-Target pairs can exist in a process instance, each adhering to this immediate succession rule. Events of Target can occur elsewhere in the process but not between an Activation and its corresponding Target.

Constraint Template: Chain Succession[Activation, Target]
Semantics: Whenever an Activation event occurs, it must be immediately followed by a Target event, with no other events in between. Conversely, whenever a Target event occurs, it must be immediately preceded by an Activation event, with no other events in between. This enforces a strict one-to-one and immediate pairing of Activations and Targets, ensuring that neither occurs without the other and that they occur consecutively without interruption.

Constraint Template: Co-Existence[Activation, Target]
Semantics: If Activation occurs in a process instance, then Target must also occur in the same process instance, and vice versa. This mutual obligation ensures that neither event can occur without the other. The events can occur multiple times and in any order relative to each other. If neither event occurs, the constraint is still considered satisfied.

Constraint Template: Precedence[Target, Activation]
Semantics: An Activation event is permitted to occur only if a Target event has occurred before it within the same process instance. Activations that occur without any prior Target are violations of the constraint. Multiple Targets can occur, and any number of events (including other Activations) can happen between a Target and a subsequent Activation. This ensures that certain actions cannot proceed without the necessary prerequisites being met first.

Constraint Template: Responded Existence[Activation, Target]
Semantics: If Activation occurs in a process instance, then Target must also occur at least once within the same process instance, either before or after the Activation. If Activation does not occur, there is no requirement for Target to occur. Multiple occurrences of Activation are satisfied by at least one occurrence of Target, and multiple Targets are allowed.

Constraint Template: Response[Activation, Target]
Semantics: If an Activation occurs, then a Target must eventually occur after the Activation at some point in the same process instance. There is no requirement on how soon or what events may occur in between. Multiple Activations can be pending at the same time, and a single Target can fulfill the obligation for all preceding Activations. If Activation does not occur, the constraint imposes no obligation on the occurrence of Target.

Constraint Template: Succession[Activation, Target]
Semantics: An Activation event must be followed by a Target event at some point later in the same process instance. Conversely, a Target event can occur only if an Activation event has occurred before it within the same process instance. This bidirectional and ordered dependency ensures that Activations impose obligations for future Targets, and Targets cannot occur without prior Activations. Multiple Activations and Targets are allowed, with each Activation eventually leading to a Target and each Target being justified by a preceding Activation..
</Binary Positive Templates>

####**Binary Negative Templates**

<Binary Negative Templates>
Constraint Template: Not Chain Succession[Activation, Target]
Semantics: An Activation event cannot be immediately followed by a Target event; there must be at least one other event between them. Similarly, a Target event cannot be immediately preceded by an Activation event. This constraint allows multiple occurrences of Activations and Targets in any order, provided they are not adjacent in the event sequence.

Constraint Template: Not Co-Existence[Activation, Target]
Semantics: The Activation event and the Target event cannot both occur within the same process instance. This means that if Activation occurs one or more times, Target must not occur at all in that process instance, and vice versa. It is acceptable for neither event to occur. The constraint enforces a mutual exclusion between Activation and Target, allowing for different process variants based on which event (if any) occurs.

Constraint Template: Not Succession[Activation, Target]
Semantics: An Activation event and a Target event must not occur such that the Activation is followed by the Target at any point later in the same process instance. Similarly, a Target event must not be preceded by an Activation event. This means that once an Activation occurs, Target cannot occur afterward, and if a Target occurs, it cannot have been preceded by an Activation. Both events can occur in the same process instance, but not in the prohibited sequence.

Constraint Template: Not Chain Precedence[Target, Activation]
Semantics: A Target event cannot be directly preceded by an Activation event, meaning that a Target cannot occur immediately after an Activation without any intervening events.

Constraint Template: Not Chain Response[Activation, Target]
Semantics: An Activation event cannot be directly followed by a Target event, meaning that a Target cannot occur immediately after an Activation without any intervening events.

Constraint Template: Not Precedence[Target, Activation]
Semantics: An Activation event cannot be preceded by a Target event within the same process instance. This means that if any Target event has occurred before an Activation event in the process instance, then that Activation is disallowed. Multiple Activations and Targets can occur, but all instances of the Activation must occur before any instance of the Target.

Constraint Template: Not Responded Existence[Activation, Target]
Semantics: If an Activation event occurs in a process instance, then a Target event must not occur anywhere in the same process instance after the Activation. This means that the occurrence of Activation prohibits any subsequent occurrence of Target. However, if Activation does not occur, Target may occur freely. Multiple occurrences of Activation are allowed, provided Target does not occur at all in the process instance.

Constraint Template: Not Response[Activation, Target]
Semantics: An Activation event must not be followed by a Target event at any point later in the same process instance. This means that once an Activation occurs, Target cannot occur afterward. However, Target events occurring before the Activation are permitted. The constraint ensures that after the occurrence of Activation, the process does not allow any further occurrences of Target.
</Binary Negative Templates>

####**Choice Templates**

<Choice Templates>
Constraint Template: Choice[Activation, Target]
Semantics: At least one of the activities Activation or Target must occur in the process instance. This means that the process is compliant if Activation occurs, Target occurs, or both occur. There is no restriction on the order or the number of times they occur. However, if neither Activation nor Target occurs, the constraint is violated.

Constraint Template: Exclusive Choice[Activation, Target]
Semantics: Exactly one of the activities Activation or Target must occur in the process instance. This means that the process is compliant if only Activation occurs without any occurrence of Target, or only Target occurs without any occurrence of Activation. The process is non-compliant if both Activation and Target occur, or if neither occurs. The constraint enforces a mutual exclusivity and a mandatory occurrence of one of the specified activities.
</Choice Templates>

##**Conditions**
<Conditions>
There are three types of conditions in MP-Declare: activation, correlation and time.
<Activation and Correlation>
###**Activation and Correlation:**
<Activation Condition Example>
A.Phone Number
</Activation Condition Example>
<Correlation Condition Example>
T.Phone Number
</Correlation Condition Example>

###**Conditions Operations Guidelines**
<Conditions Operations Guidelines>
Conditions have operations on attributes and should always reference existing attributes values that are bound to the activities of the condition.

<Enumeration Operations>Operations on attribute values for enumeration: is, is not, in, not in.</Enumeration Operations>

**Examples of Operations in Enumeration Attributes:**
<Enumeration Operations Examples>
A.Transport Type is Car
A.Transport Type is not Car
A.Transport Type in (Car, Train)
A.Transport Type not in (Car, Train)

If both activation condition and correlation condition have the same attribute, == can be specified for equals, and != can be specified as different in the correlation condition section.

**Example for ==:**
<Example double equals operator>A.Transport Type == T.Transport Type</Example double equals operator>
**Example for or !=:**
<Example not equal to operator>A.Transport Type != T.Transport Type</Example not equal to operator>
</Enumeration Operations Examples>

<Integer or Float Operations>Operations on attribute values for numeric (integer or float) attribute: <, <=, =, =>, >.</Integer or Float Operations>
**Examples of Operations in Float or Integer Attributes:**
<Integer or Float Operations Examples>
A.Price > 10
A.Price <= 5
A.Price == 3
</Integer or Float Operations Examples>

<Logical Operations>Operations can be joined with 'and' and 'or'.</Logical Operations>
<Logical Operations Examples>
(T.Price <= 10) OR (T.Price > 100)
(T.Price = 10) AND (T.Transport Type is Car)
</Logical Operations Examples>

</Conditions Operations Guidelines>
</Activation and Correlation>

<Time Condition Guidelines>
The time condition must be specified as follows: min_value, max_value, measure. Where min_value and max_value are positive integers and min_value<=max_value and measure refers to the measure of time which can be s for seconds, m for minutes, h for hours or d for days.
</Time Condition Guidelines>

**Examples of Time Conditions Specification:**
<Time Condition Examples>
10,15,s: means that the condition must be triggered between 10 and 15 seconds.
5,13,m: means that the condition must be triggered between 5 and 13 minutes.
0,48,h: means that the condition must be triggered in at most 48 hours.
0,2,d: means that the condition must be triggered in at most 2 days.
</Time Condition Examples>
</Conditions>

Examples of model descriptions are provided below to guide your output and ensure alignment between data representation and text description. Your output will be the name of the construct and generated description.

###Activity Examples
####1
Name: activity Plan Event
Description:The 'Plan Event' activity represents the planning phase for organizing an event.
####2
Name: activity Collect Feedback
Description: The 'Collect Feedback' activity includes gathering feedback from guests and staff about the event.
####3
Name: activity Coordinate Photography
Description: The 'Coordinate Photography' activity involves managing photography services to capture moments during the event.

###Attributes Exampels
####1
Name: User ID: Integer between 1 and 1000
Description: The attribute 'User ID: Integer between 1 and 1000' represents the user's identification number for authentication purposes.
####2
Name: User Password: Float between 1000000 and 9000000
Description: The attribute 'User Password: Float between 1000000 and 9000000' represents the user's password for login.
####3
Name: Product Type: Cloths, radio, computer, notebook, food, chocolate
Description: The attribute 'Product Type: cloths, radio, computer, notebook, food, chocolate' specifies the types of products available for purchase.

###Binds Examples
####1
Name: bind Driving Test: Driver, Grade
Description: The bind 'Driving Test: Driver, Grade' specifies that the activity 'Driving Test' requires the attributes 'Driver' and 'Grade'.
####2
Name: bind Book Venue: Event ID, Venue
Description: The bind 'Book Venue: Event ID, Venue' links the 'Book Venue' activity with the attributes 'Event ID' and 'Venue'.
####3
Name: bind Hire Staff: Event ID, Staff Role
Description: The bind 'Hire Staff: Event ID, Staff Role' links the 'Hire Staff' activity with the attributes 'Event ID' and 'Staff Role'.

###Constraints Examples
####1
Name: Init[Plan Event] | |
Description: The 'Plan Event' must be the first activity to occur in the process.
####2
Name: End[Collect Feedback] | |
Description: The 'Collect Feedback' must be the last activity to occur in the process.
####3
Name: Existence[Conduct Internal Audit] | |
Description: The 'Conduct Internal Audit' activity must occur at least once in the process.
####4
Name: Existence4[Declare Contigency] | |
Description: The 'Declare Contigency' activity must occur at least 4 times in the process.
####5
Name: Absence[Stop Process] | |
Description: The 'Stop Process' must not occur in the process.
####6
Name: Absence3[Retry Access] | |
Description: The 'Retry Access' must not occur 3 times in the process.
####7:
Name: Exactly[Send Message] | |
Description: The 'Send Message' activity must occur exactly once in the process.
####8:
Name: Exactly5[Get Medicines] | |
Description: The 'Get Medicines' activity must occur exactly 5 times in the process.
####9:
Name: Alternate Precedence[Pay Bill, Get Goods] | | |
Description: The 'Get Goods' activity can occur only if it was immediately preceded by a 'Pay Bill' activity, and no other 'Pay Bill' event is allowed between the 'Pay Bill' and the 'Get Goods'.
####10:
Name: Alternate Response[Walk in the Park, Drink Water]
Description: If 'Walk in the Park' activity occurs, then 'Drink Water' activity  must occur after that 'Walk in the Park' event and before any subsequent 'Walk in the Park' occurs no other occurrence of 'Walk in the Park'is allowed between an 'Walk in the Park' and 'Drink Water'.
####11:
Name: Alternate Succession[Open Browser, Login] | | |
Description: Each occurrence of a 'Open Browser' activity must eventually be followed by a 'Login' activity, and each occurrence of a 'Login' must be preceded by 'Open Browser', with no intermediate occurrences of 'Open Browser' or 'Login' between them.
####12:
Name: Chain Precedence[Finish Process, Review Process] | | |
Description: The 'Finish Process' activity can occur only if it is immediately preceded by 'Review Process' activity, with no other activities occurring in between.
####13:
Name: Chain Response[Buy Medicine, Recover] | | |
Description: Whenever a 'Buy Medicine' activity occurs, it must be immediately followed by a 'Recover' activity in the next event position, with no other events in between. The 'Recover' activity is allowed to happen isolated.
####14:
Name: Chain Succession[Open the Door, Close the Door] | | |
Description: Whenever a 'Open the Door' activity occurs, it must be immediately followed by a 'Close the Door' activity, with no other events in between. Conversely, whenever a 'Close the Door' event occurs, it must be immediately preceded by a 'Open the Door' event, with no other events in between.
####15:
Name: Co-Existence[Send Inquiry, Execute Demand] | | |
Description: If 'Send Inquiry' activity occurs, then 'Execute Demand' must also occur in the same process instance, and vice-versa.
####16:
Name: Precedence[Serve Meals, Deliver Speech] | | |
Description: The 'Serve Meals' activity is permitted to occur only if a 'Deliver Speech' activity has occurred before it within the same process instance.
####17:
Name: Responded Existence[Login, Logout] | | |
Description: If 'Login' activity occurs in a process instance, then 'Logout' activity must also occur at least once within the same process instance, either before or after the 'Login' event. If 'Login' does not occur, there is no requirement for 'Logout' to occur. Multiple occurrences of 'Login' are satisfied by at least one occurrence of 'Logout', and multiple 'Logout' instances are allowed.
####18:
Name: Response[Open Browser, Fill URL] | | |
Description: If a 'Open Browser' activity occurs, then a 'Fill URL' activity must eventually occur after the 'Open Browser' event at some point in the same process instance. There is no requirement on how soon or what events may occur in between. Multiple 'Open Browser' can be pending at the same time, and a single 'Fill URL' can fulfill the obligation for all preceding 'Open Browser'. If 'Open Browser' does not occur, the constraint imposes no obligation on the occurrence of 'Fill URL'.
####19:
Name: Succession[Open Umbrella, Close Umbrella] | | |
Description: A 'Open Umbrella' activity must be followed by a 'Close Umbrella' activity at some point later in the same process instance. Conversely, a 'Close Umbrella' event can occur only if an 'Open Umbrella' event has occurred before it within the same process instance. This bidirectional and ordered dependency ensures that 'Open Umbrella' impose obligations for future 'Close Umbrella', and 'Close Umbrella' cannot occur without prior 'Open Umbrella'. Multiple 'Open Umbrella' and 'Close Umbrella' are allowed, with each 'Open Umbrella' eventually leading to a 'Close Umbrella' and each 'Close Umbrella' being justified by a preceding 'Open Umbrella'.
####20:
Name: Not Chain Succession[Style Hair, Take a Shower] | | |
Description: A 'Style Hair' activity cannot be immediately followed by a 'Take a Shower' activity; there must be at least one other event between them. Similarly, a 'Take a Shower' event cannot be immediately preceded by a 'Style Hair' event. This constraint allows multiple occurrences of 'Style Hair' and 'Take a Shower' in any order, provided they are not adjacent in the event sequence.
####21:
Name: Not Co-Existence[Go to Party, Stay at Home] | | |
Description: The 'Go to Party' activity and the 'Stay at Home' activity cannot both occur within the same process instance. This means that if 'Go to Party' event occurs one or more times, 'Stay at Home' must not occur at all in that process instance, and vice-versa. It is acceptable for neither event to occur. The constraint enforces a mutual exclusion between 'Go to Party' and 'Stay at Home', allowing for different process variants based on which event (if any) occurs.
####22:
Name: Not Succession[Eat Food, Run] | | |
Description: The 'Eat Food' activity and 'Run' activity must not occur such that the 'Eat Food' event is followed by the 'Run' event at any point later in the same process instance. Similarly, a 'Run' event must not be preceded by an 'Eat Food' event. This means that once an 'Eat Food' occurs, 'Run' cannot occur afterward, and if 'Run' event occurs, it cannot have been preceded by an 'Eat Food' event. Both events can occur in the same process instance, but not in the prohibited sequence.
####23:
Name: Not Chain Precedence[Order Confirmation, Order Placement] | | |
Description: The 'Order Confirmation' activity cannot be directly preceded by an 'Order Placement' activity, meaning that a 'Order Confirmation' event cannot occur immediately after an 'Order Placement' without any intervening events.
####24:
Name: Not Chain Response[Submit Order, Confirm Order] | | |
Description: The 'Submit Order' activity cannot be directly followed by a 'Confirm Order' activity, meaning that a 'Confirm Order' event cannot occur immediately after a 'Submit Order' without any intervening events.
####25:
Name: Not Precedence[Approve Order, Process Payment] | | |
Description: A 'Process Payment' activity cannot be preceded by a 'Approve Order' within the same process instance. If a 'Approve Order' event has occurred at any point before a 'Process Payment' event, then 'Process Payment' is disallowed. Multiple instances of 'Process Payment' and 'Approve Order' can occur, but all instances of 'Process Payment' must occur before any instance of 'Approve Order'.
####26:
Name: Not Responded Existence[Submit Order, Cancel Order] | | |
Description: If a 'Submit Order' activity occur in a process instance, then a 'Cancel Order' activity must not occur anywhere in the same process instance after the 'Submit Order' event. This means that the occurrence of 'Submit Order' event prohibits any subsequent occurrence of 'Cancel Order' event. However, if 'Submit Order' does not occur, 'Cancel Order' may occur freely. Multiple occurrences of 'Submit Order' are allowed, provided 'Cancel Order' does not occur at all in the process instance.
####27:
Constraint Template: Not Response[Submit Application, Reject Application] | | |
Description: The 'Submit Application' activity must not be followed by a 'Reject Application' activity at any point later in the same process instance. This means that once a 'Submit Application' event occurs, 'Reject Application' cannot occur afterward. However, 'Reject Application' events occurring before the 'Submit Application' are permitted. The constraint ensures that after the occurrence of 'Submit Application', the process does not allow any further occurrences of 'Reject Application'.
####28:
Name: Choice[Receive Payment, Issue Refund] | | |
Description: At least one of the activities 'Receive Payment' or 'Issue Refund' must occur in the process instance. This means that the process is compliant if 'Receive Payment' event occurs, 'Issue Refund' event occurs, or both occur. There is no restriction on the order or the number of times they occur. However, if neither 'Receive Payment' nor 'Issue Refund' occurs, the constraint is violated.
####29:
Name: Exclusive Choice[Approve Application, Reject Application] | | |
Description: Exactly one of the activities 'Approve Application' or 'Reject Application' must occur in the process instance. This means that the process is compliant if only 'Approve Application' event occurs without any occurrence of 'Reject Application' event, or only 'Reject Application' occurs without any occurrence of 'Approve Application'. The process is non-compliant if both 'Approve Application' and 'Reject Application' occur, or if neither occurs. The constraint enforces a mutual exclusivity and a mandatory occurrence of one of the specified activities.
<\Assistant Prompt>
"""

# 4 functions to get model description.
toolsdescription = [
  {
    "type": "function",
    "function": {
      "name": "get_activities_description",
      "description": "Fetch the activities of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "activities": {
            "type": "array",
            "description": "The activities contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the activity."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the activity."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_attributes_description",
      "description": "Fetch the attributes of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "attributes": {
            "type": "array",
            "description": "The attributes contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the attribute."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the attribute."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_binds_description",
      "description": "Fetch the binds of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "binds": {
            "type": "array",
            "description": "The binds contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the bind."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the bind."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_constraints_description",
      "description": "Fetch the constraints of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "constraints": {
            "type": "array",
            "description": "The constraints contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the constraint."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the constraint."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  }
]


##MP-Declare Data Model Schema

The following data model were used to validate if MP-Declare models are compliant with Declare4Py specifications.

In [ ]:
# Structured Output Data Schema
class Activity(BaseModel):
    """
    Represents an activity in a process model.

    Attributes:
        name (str): The name of the activity.
        description (str): A detailed description of what the activity entails.
    """
    name: str = Field(
        ...,
        description="The name of the activity. It's represented by an action in the process.",
    )
    description: str = Field(
        ...,
        description="A detailed description of what the activity entails.",
    )

class Attribute(BaseModel):
    """
    Represents an attribute associated with an activity.

    Attributes:
        type (str): The type of attribute. It can be integer, float or enumeration.
        name (str): The name of the attribute.
        description (str): A detailed description of what the attribute represents.
        min_value (Optional[Union[float, int]]): Minimum value for integer or float attributes.
        max_value (Optional[Union[float, int]]): Maximum value for integer or float attributes.
        enumeration_values (Optional[List[str]]): Possible values for enumeration attributes.
    """
    type: Literal["integer", "float", "enumeration"] = Field(
        ...,
        description="The type of attribute. It can be integer, float or enumeration.",
    )
    name: str = Field(
        ...,
        description="The name of the attribute.",
    )
    description: str = Field(
        ...,
        description="A detailed description of what the attribute represents.",
    )
    min_value: Optional[Union[float, int]] = Field(
        None,
        description="Minimum value for integer or float attributes. Should be <= than max_value."
    )
    max_value: Optional[Union[float, int]] = Field(
        None,
        description="Maximum value for integer or float attributes. Should be >= than min_value."
    )
    enumeration_values: Optional[List[str]] = Field(
        None,
        description="Possible values for enumeration attributes."
    )

    @field_validator('description', mode='before')
    def validate_description(cls, value):
        """
        Ensure description is not empty.

        Raises:
            ValueError: If description is empty.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating description for attribute: {value}")
        if not value:
            raise ValueError("Description must not be empty.")
        return value

    @field_validator('min_value', 'max_value', mode='before')
    def check_min_max_values(cls, value, info):
        """
        Validate min_value and max_value based on the type of attribute.

        Raises:
            ValueError: If min_value or max_value are required but not provided.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating {info.field_name} for attribute type {info.data['type']}.")
        if info.data['type'] == 'integer':
            if value is not None:
                value = int(value)
            else:
                raise ValueError(f"{info.field_name} is required for {info.data['type']} type.")
        elif info.data['type'] == 'float':
            if value is None:
                raise ValueError(f"{info.field_name} is required for {info.data['type']} type.")
        return value

    @field_validator('min_value', 'max_value', mode='after')
    def ensure_min_less_than_max(cls, value, info):
        """
        Ensure that min_value is less than or equal to max_value.

        Raises:
            ValueError: If min_value is greater than max_value.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Ensuring {info.field_name} for attribute type {info.data['type']} is within valid range.")
        if info.data.get('min_value') is not None and info.data.get('max_value') is not None:
            if info.data['min_value'] > info.data['max_value']:
                raise ValueError("min_value must be less than or equal to max_value.")
        return value

    @field_validator('enumeration_values', mode='before')
    def check_enumeration_values(cls, value, info):
        """
        Validate enumeration_values for enumeration type attributes.

        Raises:
            ValueError: If enumeration_values has fewer than 1 unique items.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating enumeration values for attribute type {info.data['type']}.")
        if info.data['type'] == 'enumeration':
            if not value or len(value) < 1:
                raise ValueError("enumeration_values must have at least 1 unique item for enumeration type.")
        return value

class Bind(BaseModel):
    """
    Represents the binding of attributes to an activity.

    Attributes:
        activity (Activity): The activity involved in the binding.
        attributes (List[Attribute]): List of attribute objects bound to the activity.
        description (str): Description of the purpose or role of the bind.
    """
    activity: Activity = Field(
        ...,
        description="The activity involved in the binding."
    )
    attributes: conlist(Attribute) = Field(
        ...,
        description="List of attribute objects bound to the activity. The attribute must have the min_value and max_value if it is integer or float and if it is enumeration it must have at least one option."
    )
    description: str = Field(
        ...,
        description="Description of the purpose or role of the bind."
    )

    @model_validator(mode='before')
    def validate_attributes(cls, values):
        """
        Ensure attributes are valid instances of Attribute.

        Raises:
            ValueError: If attributes are not valid instances of Attribute.

        Returns:
            values: The validated values.
        """
        logger.debug(f"Validating attributes for bind: {values}")
        attributes = values.get('attributes', [])
        for attribute in attributes:
            if not isinstance(attribute, Attribute):
                Attribute(**attribute)
        return values

    @field_validator('description', mode='before')
    def validate_description(cls, value):
        """
        Ensure description is not empty.

        Raises:
            ValueError: If description is empty.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating description for bind: {value}")
        if not value:
            raise ValueError("Description must not be empty.")
        return value

class Constraint(BaseModel):
    """
    Represents a constraint in the process model.

    Attributes:
        type (str): The type of constraint. It can be unary or binary.
        description (str): A detailed description of what the constraint entails.
        template (str): The template of the constraint.
        activation (Optional[Activity]): The activation activity involved in the constraint.
        target (Optional[Activity]): The target activity involved in the constraint.
        activation_condition (Optional[str]): Condition to activate the constraint.
        correlation_condition (Optional[str]): Condition to correlate the constraint.
        time_condition (Optional[str]): Condition to specify time constraints.
        cardinality (Optional[int]): The cardinality for the Existence, Absence, and Exactly templates.
        all_binds (Dict[str, Bind]): Dictionary of binds associated with the constraint.
    """
    type: Literal["unary", "binary"] = Field(
        ...,
        description="The type of constraint. It can be unary or binary. Unary templates have only activation and binary templates must have activation and target.",
    )
    description: str = Field(
        ...,
        description="A detailed description of what the constraint entails.The time conditions should always be described in the original measure."
    )
    template: Literal["Init", "End", "Existence", "Absence", "Exactly", "Alternate Precedence", "Alternate Response", "Alternate Succession", "Chain Precedence", "Chain Response", "Chain Succession", "Co-Existence", "Precedence", "Responded Existence", "Response", "Succession", "Not Chain Succession", "Not Co-Existence", "Not Succession", "Not Chain Precedence", "Not Chain Response", "Not Precedence", "Not Responded Existence", "Not Response", "Choice", "Exclusive Choice"] = Field(
        ...,
        description="The template of the constraint. If the type of the constraint is unary, the following templates are applicable: Init, End, Existence, Absence, Exactly. If it is binary template, the following are applicable: Alternate Precedence, Alternate Response, Alternate Succession, Chain Precedence, Chain Response, Chain Succession, Co-Existence, Precedence, Responded Existence, Response, Succession, Not Chain Succession, Not Co-Existence, Not Succession, Not Chain Precedence, Not Chain Response, Not Precedence, Not Responded Existence, Not Response, Choice, Exclusive Choice ",
    )
    activation: Activity = Field(
        ...,
        description="The activation activity involved in the constraint. An activation activity is the event or activity that triggers a specific constraint or rule in the process model. It is the starting point or the condition that needs to be met for a constraint to be evaluated."
    )
    target: Optional[Activity] = Field(
        None,
        description="The target activity involved in the constraint. A target activity is the event or activity that is affected or constrained by the activation activity. It is the activity whose occurrence, timing, or order is governed by the constraint that has been activated. Its mandatory that binary templates have target activity."
    )
    activation_condition: Optional[str] = Field(
        None,
        description="Condition to activate the constraint. It must reference attributes of the activation activity and may use operations in, not in, is, is not for enumeration attributes. For integer and float it can use the following operators: ==, !=, >, >=, <, <= . And conditions can be joined using AND and OR.",
    )
    correlation_condition: Optional[str] = Field(
        None,
        description="Condition to correlate the activation and target of the constraint. It must reference attributes of both activation and target activities and may use operations in, not in, is, is not for enumeration attributes. For integer and float it can use the following operators: ==, !=, >, >=, <, <= . And conditions can be joined using AND and OR.",
    )
    time_condition: Optional[str] = Field(
        None,
        description="Condition to specify time of the activation of the constraint. It can be in seconds (s), minutes (m), hours (h) or days (d).",
    )
    cardinality: Optional[int] = Field(
        None,
        description="The cardinality that is mandatory for some unary templates: Existence, Absence, and Exactly."
    )

    @model_validator(mode='before')
    def validate_template_type_match(cls, values):
        """
        Ensure the template matches the type of the constraint.

        Args:
            values (dict): The values to validate.

        Raises:
            ValueError: If the template is not valid for the given type.

        Returns:
            dict: The validated values.
        """
        constraint_type = values.get('type')
        template = values.get('template')

        unary_templates = ["Init", "End", "Existence", "Absence", "Exactly"]
        binary_templates = ["Alternate Precedence", "Alternate Response", "Alternate Succession", "Chain Precedence", "Chain Response", "Chain Succession", "Co-Existence", "Precedence", "Responded Existence", "Response", "Succession", "Not Chain Succession", "Not Co-Existence", "Not Succession", "Not Chain Precedence", "Not Chain Response", "Not Precedence", "Not Responded Existence", "Not Response", "Choice", "Exclusive Choice"]

        logger.debug(f"Validating constraint: type={constraint_type}, template={template}")

        if constraint_type == "unary" and template not in unary_templates:
            raise ValueError(f"Invalid template '{template}' for unary constraint")
        if constraint_type == "binary" and template not in binary_templates:
            raise ValueError(f"Invalid template '{template}' for binary constraint")
        return values

    @field_validator('activation', 'target', mode='before')
    def check_activation_and_target(cls, value, info):
        """
        Validate activation and target based on the type of constraint.

        Raises:
            ValueError: If activation or target is required but not provided.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating {info.field_name} for constraint type {info.data['type']}.")
        if info.data['type'] == 'binary':
            if info.field_name == 'activation' and value is None:
                raise ValueError("activation is required for binary type constraints.")
            if info.field_name == 'target' and value is None:
                raise ValueError("target is required for binary type constraints.")
        if info.data['type'] == 'unary' and info.field_name == 'activation' and value is None:
            raise ValueError("activation is required for unary type constraints.")
        return value



    @field_validator('cardinality', mode='before')
    def check_cardinality(cls, value, info):
        """
        Validate cardinality for Existence, Absence and Exactly templates.

        Raises:
            ValueError: If cardinality is required but not provided.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating cardinality for template {info.data['template']}.")
        if info.data['template'] in ['Existence', 'Absence', 'Exactly'] and value is None:
            raise ValueError("cardinality is required for Existence, Absence, and Exactly templates.")
        return value


class MPDeclareModel(BaseModel):
    """
    Represents a process model with activities, attributes, binds, and constraints.

    Attributes:
        activities (List[Activity]): List of activities in the process model.
        attributes (List[Attribute]): List of attributes used in the process model.
        binds (List[Bind]): List of binds that links activities and attributes used in the process model.
        constraints (List[Constraint]): List of constraints that rule the process execution.
    """
    activities: List[Activity] = Field(
        ...,
        description="List of activities used in the process model.",
    )
    attributes: List[Attribute] = Field(
        ...,
        description="List of attributes used in the process model.",
    )
    binds: List[Bind] = Field(
        ...,
        description="List of binds that links activities and attributes used in the process model.",
    )
    constraints: List[Constraint] = Field(
        ...,
        description="List of constraints that rule the process execution.",
    )

    def convert_to_string(self) -> str:
        """
        Convert the process model to a string representation.

        Returns:
            str: The string representation of the process model.
        """
        # Convert activities
        activities_str = "\n".join(
            [f"activity {activity.name}" for activity in self.activities]
        )

        # Convert binds
        binds_str = "\n".join(
            [
                f"bind {bind.activity.name}: " + ", ".join([attr.name for attr in bind.attributes])
                for bind in self.binds
            ]
        )

        # Convert attributes
        attributes_str = ""
        for attribute in self.attributes:
            if attribute.type in ['integer', 'float']:
                attributes_str += f"{attribute.name}: {attribute.type} between {attribute.min_value} and {attribute.max_value}\n"
            elif attribute.type == 'enumeration':
                enumeration_values = ", ".join(attribute.enumeration_values)
                attributes_str += f"{attribute.name}: {enumeration_values}\n"

        # Convert constraints
        constraints_str = ""
        for constraint in self.constraints:
            constraint_str = ""  # Initialize constraint_str at the start of the loop
            if constraint.type == 'unary':
                if constraint.cardinality and constraint.cardinality > 1:
                    constraint_str = f"{constraint.template}{constraint.cardinality}[{constraint.activation.name}]"
                else:
                    constraint_str = f"{constraint.template}[{constraint.activation.name}]"
                constraint_str += f" |{constraint.activation_condition or ''} |{constraint.time_condition or ''}"
            elif constraint.type == 'binary':
                constraint_str = f"{constraint.template}[{constraint.activation.name}, {constraint.target.name}]"
                constraint_str += f" |{constraint.activation_condition or ''} |{constraint.correlation_condition or ''} |{constraint.time_condition or ''}"
            constraints_str += f"{constraint_str}\n"
        # Combine all parts
        final_str = "\n".join([activities_str, binds_str, attributes_str.strip(), constraints_str.strip()])

        return final_str


#Generation Flow

##Declare4py Object Instantiation

In [ ]:
model ="""
"""

In [ ]:
#Instanciating the ExtendedDeclareModel with the string of the MPDeclareModel.
declare_model = ExtendedDeclareModel().parse_from_string(model)

# Fetching activities, constraints, attributes, and binds from the model
model_activities = declare_model.get_model_activities()
model_attributes = declare_model.get_decl_attributes()
model_binds = declare_model.get_decl_binds()
model_constraints = declare_model.get_decl_model_constraints()
print("MP-Declare Model Constructs")
# Printing model activities
print("Model activities:")
print("-----------------")
for idx, act in enumerate(model_activities):
    print(idx, act)
print("\n")

# Printing model attributes
print("Model attributes:")
print("-----------------")
for idx, attr in enumerate(model_attributes):
    print(idx, attr)
print("\n")

# Printing model binds
print("Model binds:")
print("-------------")
for idx, bind in enumerate(model_binds):
    print(idx, bind)
print("\n")

# Printing model constraints
print("Model constraints:")
print("-----------------")
for idx, constr in enumerate(model_constraints):
    print(idx, constr)
print("\n")


##MP-Declare to Intermediary Description Generation

The goal of this step is to generate intermediary descriptions of MP-Declare constructs. This is essential to have semantically enriched descriptions which can be used to evaluate if each part of the model is covered semantically and with correctness and completeness.

To achieve a consistent format of the intermediary descriptions we used some prompt engineering techniques: role-play prompting, knowledge infusion, few-shot learning and instruction tunning. Prompt engineering leverages large language models' capabilities without no need to fine-tune the model to achieve a goal.

In [ ]:
# The main prompt used to generate the intermediary descriptions.
decltotextassistant="""
<Assistant Prompt>
You are an an expert in declarative business process management, specifically for interpreting and describing Multi-Perspective Declare models (MP-Declare). Your primary task is to analyze the provided MP-Declare model and generate a comprehensive and consistent description. This description should meticulously reflect the structure and terminology of the model, maintaining exact naming and case sensitivity for model parts (activities, attributes, binds, and constraints).
Use the following knowledge base to know how to interpret the MP-Declare model:
<MP-Declare Model Specification>
#**Multi-Perspective Declare (MP-Declare) Constructs**

##**Activities**

<Activities Specification Guidelines>
Activities are specified with the word 'activity' followed by descriptive names, which should use active verbs, avoid abbreviations, and follow naming conventions (i.e., <Action Verb> <Object> or <Object> <Action Verb>).
</Activities Specification Guidelines>

**Examples of activities specification:**
<Activities Examples>
activity Driving Test
activity Initiate Purchase Order
</Activities Examples>
</Activities Specification Guidelines>

##**Binds**

<Binds Specification Guidelines>
Binds are specified as a link between activities and attributes. They have as a prefix the activity name followed by ':' and then a list of attributes, each separated by ','.

**Examples of binds specification:**
<Binds Examples>
Driving Test: Driver, Grade
Initiate Purchase Order: Purchase ID, Customer ID, Order Status, Order Value
</Binds Examples>
</Binds Specification Guidelines>

##**Attributes**
<Attributes Specification Guidelines>
MP-Declare models supports three types of attributes: integer, float and enumeration.

**Attribute Types:**
- **Integer:** Represents numeric integer values within a specified range.
- **Float:** Represents numeric values with floating-point within a specified range.
- **Enumeration:** Represents a limited set of possible values.

**Integer specification:**<Integer Attribute Specification Guidelines>[{Attribute Name}: integer between {min_value} and {max_value}], where {attribute name} is the name of attribute and {min_value} and {max_value} are two integer numbers.</Integer Attribute Specification Guideline>
**Float specification:**<Float Attribute Specification Guideline>[{Attribute Name}: float between {min_value} and {max_value}], where {attribute name} is the name of attribute and {min_value} and {max_value} are two float numbers.</Float Attribute Specification Guideline>
**Enumeration specification:**<Enumeration Specification Guideline>[{Attribute Name}: {value1}, {value2}, ..., {valuen}], where {attribute name} is the name of attribute and {value1} to {valuen} represent possible values for the attribute. These possible values should always be at least 2 and not infinite.</Enumeration Specification Guidelines>

**Examples of attributes specification:**
<Attributes Examples>
**Integer attribute example:** <Integer Attribute Example>Grade: integer between 1 and 5</Integer Attribute Example>.
**Float attribute example:** <Float Attribute Example>Order Value: float between 1.99 and 99999.9</Float Attribute Example>.
**Enumeration attribute example:** <Enumeration Attribute Example>Order Status: Approved, Not Approved, Shipped, Not Shipped</Enumeration Attribute Example>.
</Attributes Examples>
</Attributes Specification Guidelines>

##**Constraints**

MP-Declare has two types of templates: unary and binary. Following is specified the templates and their semantics.

###**Unary Templates**

Unary templates have only Activation activity.

<Unary Constraints Templates>
Constraint Template: Absence[n][Activation]
Semantics: Activation must not occur n times in the process.

Constraint Template: Exactly[n][Activation]
Semantics: Activation must occur exactly n times in the process.

Constraint Template: Existence[n][Activation]
Semantics: Activation must occur at least n times in the process.

Constraint Template: Init[Activation]
Semantics: Activation must be the first activity to occur in the process.

Constraint Template: End[Activation]
Semantics: Activation must be the last activity to occur in the process.
</Unary Constraints Templates>

###**Binary Templates**
Binary templates have both Activation and Target activities.
####**Binary Positive Templates**

<Binary Positive Templates>
Constraint Template: Alternate Precedence[Target, Activation]
Semantics: An Activation event can occur only if it was immediately preceded by a Target event, and no other Target event is allowed between that Target and the Activation. This ensures a one-to-one and ordered pairing between Targets and Activations, preventing any Activation from occurring without the necessary prior Target and avoiding overlapping or multiple Targets before an Activation.

Constraint Template: Alternate Response[Activation, Target]
Semantics: If an Activation occurs, then a Target must occur after that Activation and before any subsequent Activation occurs no other occurrence of Activation is allowed between an Activation and its corresponding Target. This ensures that each Activation is individually responded to by a Target in sequence.

Constraint Template: Alternate Succession[Activation, Target]
Semantics: Each occurrence of an Activation must eventually be followed by a Target, and each occurrence of a Target must be preceded by an Activation, with no intermediate occurrences of Activation or Target between them. This ensures a strict one-to-one and ordered pairing between Activations and Targets, preventing any overlapping or interruption by additional Activations or Targets in the sequence.

Constraint Template: Chain Precedence[Target, Activation]
Semantics: A Target event can occur only if it is immediately preceded by an Activation event, with no other events occurring in between. There can be multiple Activation-Target pairs within the process, each adhering to this immediate precedence rule. Activations may occur without being followed by a Target, but Targets cannot occur without such an immediate preceding Activation.

Constraint Template: Chain Response[Activation, Target]
Semantics: Whenever an Activation occurs, it must be immediately followed by a Target in the next event position, with no other events in between. This immediate succession ensures a direct and uninterrupted transition from Activation to Target. Multiple Activation-Target pairs can exist in a process instance, each adhering to this immediate succession rule. Events of Target can occur elsewhere in the process but not between an Activation and its corresponding Target.

Constraint Template: Chain Succession[Activation, Target]
Semantics: Whenever an Activation event occurs, it must be immediately followed by a Target event, with no other events in between. Conversely, whenever a Target event occurs, it must be immediately preceded by an Activation event, with no other events in between. This enforces a strict one-to-one and immediate pairing of Activations and Targets, ensuring that neither occurs without the other and that they occur consecutively without interruption.

Constraint Template: Co-Existence[Activation, Target]
Semantics: If Activation occurs in a process instance, then Target must also occur in the same process instance, and vice versa. This mutual obligation ensures that neither event can occur without the other. The events can occur multiple times and in any order relative to each other. If neither event occurs, the constraint is still considered satisfied.

Constraint Template: Precedence[Target, Activation]
Semantics: An Activation event is permitted to occur only if a Target event has occurred before it within the same process instance. Activations that occur without any prior Target are violations of the constraint. Multiple Targets can occur, and any number of events (including other Activations) can happen between a Target and a subsequent Activation. This ensures that certain actions cannot proceed without the necessary prerequisites being met first.

Constraint Template: Responded Existence[Activation, Target]
Semantics: If Activation occurs in a process instance, then Target must also occur at least once within the same process instance, either before or after the Activation. If Activation does not occur, there is no requirement for Target to occur. Multiple occurrences of Activation are satisfied by at least one occurrence of Target, and multiple Targets are allowed.

Constraint Template: Response[Activation, Target]
Semantics: If an Activation occurs, then a Target must eventually occur after the Activation at some point in the same process instance. There is no requirement on how soon or what events may occur in between. Multiple Activations can be pending at the same time, and a single Target can fulfill the obligation for all preceding Activations. If Activation does not occur, the constraint imposes no obligation on the occurrence of Target.

Constraint Template: Succession[Activation, Target]
Semantics: An Activation event must be followed by a Target event at some point later in the same process instance. Conversely, a Target event can occur only if an Activation event has occurred before it within the same process instance. This bidirectional and ordered dependency ensures that Activations impose obligations for future Targets, and Targets cannot occur without prior Activations. Multiple Activations and Targets are allowed, with each Activation eventually leading to a Target and each Target being justified by a preceding Activation..
</Binary Positive Templates>

####**Binary Negative Templates**

<Binary Negative Templates>
Constraint Template: Not Chain Succession[Activation, Target]
Semantics: An Activation event cannot be immediately followed by a Target event; there must be at least one other event between them. Similarly, a Target event cannot be immediately preceded by an Activation event. This constraint allows multiple occurrences of Activations and Targets in any order, provided they are not adjacent in the event sequence.

Constraint Template: Not Co-Existence[Activation, Target]
Semantics: The Activation event and the Target event cannot both occur within the same process instance. This means that if Activation occurs one or more times, Target must not occur at all in that process instance, and vice versa. It is acceptable for neither event to occur. The constraint enforces a mutual exclusion between Activation and Target, allowing for different process variants based on which event (if any) occurs.

Constraint Template: Not Succession[Activation, Target]
Semantics: An Activation event and a Target event must not occur such that the Activation is followed by the Target at any point later in the same process instance. Similarly, a Target event must not be preceded by an Activation event. This means that once an Activation occurs, Target cannot occur afterward, and if a Target occurs, it cannot have been preceded by an Activation. Both events can occur in the same process instance, but not in the prohibited sequence.

Constraint Template: Not Chain Precedence[Target, Activation]
Semantics: A Target event cannot be directly preceded by an Activation event, meaning that a Target cannot occur immediately after an Activation without any intervening events.

Constraint Template: Not Chain Response[Activation, Target]
Semantics: An Activation event cannot be directly followed by a Target event, meaning that a Target cannot occur immediately after an Activation without any intervening events.

Constraint Template: Not Precedence[Target, Activation]
Semantics: An Activation event cannot be preceded by a Target event within the same process instance. This means that if any Target event has occurred before an Activation event in the process instance, then that Activation is disallowed. Multiple Activations and Targets can occur, but all instances of the Activation must occur before any instance of the Target.

Constraint Template: Not Responded Existence[Activation, Target]
Semantics: If an Activation event occurs in a process instance, then a Target event must not occur anywhere in the same process instance after the Activation. This means that the occurrence of Activation prohibits any subsequent occurrence of Target. However, if Activation does not occur, Target may occur freely. Multiple occurrences of Activation are allowed, provided Target does not occur at all in the process instance.

Constraint Template: Not Response[Activation, Target]
Semantics: An Activation event must not be followed by a Target event at any point later in the same process instance. This means that once an Activation occurs, Target cannot occur afterward. However, Target events occurring before the Activation are permitted. The constraint ensures that after the occurrence of Activation, the process does not allow any further occurrences of Target.
</Binary Negative Templates>

####**Choice Templates**

<Choice Templates>
Constraint Template: Choice[Activation, Target]
Semantics: At least one of the activities Activation or Target must occur in the process instance. This means that the process is compliant if Activation occurs, Target occurs, or both occur. There is no restriction on the order or the number of times they occur. However, if neither Activation nor Target occurs, the constraint is violated.

Constraint Template: Exclusive Choice[Activation, Target]
Semantics: Exactly one of the activities Activation or Target must occur in the process instance. This means that the process is compliant if only Activation occurs without any occurrence of Target, or only Target occurs without any occurrence of Activation. The process is non-compliant if both Activation and Target occur, or if neither occurs. The constraint enforces a mutual exclusivity and a mandatory occurrence of one of the specified activities.
</Choice Templates>

##**Conditions**
<Conditions>
There are three types of conditions in MP-Declare: activation, correlation and time.
<Activation and Correlation>
###**Activation and Correlation:**
<Activation Condition Example>
A.Phone Number
</Activation Condition Example>
<Correlation Condition Example>
T.Phone Number
</Correlation Condition Example>

###**Conditions Operations Guidelines**
<Conditions Operations Guidelines>
Conditions have operations on attributes and should always reference existing attributes values that are bound to the activities of the condition.

<Enumeration Operations>Operations on attribute values for enumeration: is, is not, in, not in.</Enumeration Operations>

**Examples of Operations in Enumeration Attributes:**
<Enumeration Operations Examples>
A.Transport Type is Car
A.Transport Type is not Car
A.Transport Type in (Car, Train)
A.Transport Type not in (Car, Train)

If both activation condition and correlation condition have the same attribute, == can be specified for equals, and != can be specified as different in the correlation condition section.

**Example for ==:**
<Example double equals operator>A.Transport Type == T.Transport Type</Example double equals operator>
**Example for or !=:**
<Example not equal to operator>A.Transport Type != T.Transport Type</Example not equal to operator>
</Enumeration Operations Examples>

<Integer or Float Operations>Operations on attribute values for numeric (integer or float) attribute: <, <=, =, =>, >.</Integer or Float Operations>
**Examples of Operations in Float or Integer Attributes:**
<Integer or Float Operations Examples>
A.Price > 10
A.Price <= 5
A.Price == 3
</Integer or Float Operations Examples>

<Logical Operations>Operations can be joined with 'and' and 'or'.</Logical Operations>
<Logical Operations Examples>
(T.Price <= 10) OR (T.Price > 100)
(T.Price = 10) AND (T.Transport Type is Car)
</Logical Operations Examples>

</Conditions Operations Guidelines>
</Activation and Correlation>

<Time Condition Guidelines>
The time condition must be specified as follows: min_value, max_value, measure. Where min_value and max_value are positive integers and min_value<=max_value and measure refers to the measure of time which can be s for seconds, m for minutes, h for hours or d for days.
</Time Condition Guidelines>

**Examples of Time Conditions Specification:**
<Time Condition Examples>
10,15,s: means that the condition must be triggered between 10 and 15 seconds.
5,13,m: means that the condition must be triggered between 5 and 13 minutes.
0,48,h: means that the condition must be triggered in at most 48 hours.
0,2,d: means that the condition must be triggered in at most 2 days.
</Time Condition Examples>
</Conditions>

Examples of model descriptions are provided below to guide your output and ensure alignment between data representation and text description. Your output will be the name of the construct and generated description.

###Activity Examples
####1
Name: activity Plan Event
Description:The 'Plan Event' activity represents the planning phase for organizing an event.
####2
Name: activity Collect Feedback
Description: The 'Collect Feedback' activity includes gathering feedback from guests and staff about the event.
####3
Name: activity Coordinate Photography
Description: The 'Coordinate Photography' activity involves managing photography services to capture moments during the event.

###Attributes Exampels
####1
Name: User ID: Integer between 1 and 1000
Description: The attribute 'User ID: Integer between 1 and 1000' represents the user's identification number for authentication purposes.
####2
Name: User Password: Float between 1000000 and 9000000
Description: The attribute 'User Password: Float between 1000000 and 9000000' represents the user's password for login.
####3
Name: Product Type: Cloths, radio, computer, notebook, food, chocolate
Description: The attribute 'Product Type: cloths, radio, computer, notebook, food, chocolate' specifies the types of products available for purchase.

###Binds Examples
####1
Name: bind Driving Test: Driver, Grade
Description: The bind 'Driving Test: Driver, Grade' specifies that the activity 'Driving Test' requires the attributes 'Driver' and 'Grade'.
####2
Name: bind Book Venue: Event ID, Venue
Description: The bind 'Book Venue: Event ID, Venue' links the 'Book Venue' activity with the attributes 'Event ID' and 'Venue'.
####3
Name: bind Hire Staff: Event ID, Staff Role
Description: The bind 'Hire Staff: Event ID, Staff Role' links the 'Hire Staff' activity with the attributes 'Event ID' and 'Staff Role'.

###Constraints Examples
####1
Name: Init[Plan Event] | |
Description: The 'Plan Event' must be the first activity to occur in the process.
####2
Name: End[Collect Feedback] | |
Description: The 'Collect Feedback' must be the last activity to occur in the process.
####3
Name: Existence[Conduct Internal Audit] | |
Description: The 'Conduct Internal Audit' activity must occur at least once in the process.
####4
Name: Existence4[Declare Contigency] | |
Description: The 'Declare Contigency' activity must occur at least 4 times in the process.
####5
Name: Absence[Stop Process] | |
Description: The 'Stop Process' must not occur in the process.
####6
Name: Absence3[Retry Access] | |
Description: The 'Retry Access' must not occur 3 times in the process.
####7:
Name: Exactly[Send Message] | |
Description: The 'Send Message' activity must occur exactly once in the process.
####8:
Name: Exactly5[Get Medicines] | |
Description: The 'Get Medicines' activity must occur exactly 5 times in the process.
####9:
Name: Alternate Precedence[Pay Bill, Get Goods] | | |
Description: The 'Get Goods' activity can occur only if it was immediately preceded by a 'Pay Bill' activity, and no other 'Pay Bill' event is allowed between the 'Pay Bill' and the 'Get Goods'.
####10:
Name: Alternate Response[Walk in the Park, Drink Water]
Description: If 'Walk in the Park' activity occurs, then 'Drink Water' activity  must occur after that 'Walk in the Park' event and before any subsequent 'Walk in the Park' occurs no other occurrence of 'Walk in the Park'is allowed between an 'Walk in the Park' and 'Drink Water'.
####11:
Name: Alternate Succession[Open Browser, Login] | | |
Description: Each occurrence of a 'Open Browser' activity must eventually be followed by a 'Login' activity, and each occurrence of a 'Login' must be preceded by 'Open Browser', with no intermediate occurrences of 'Open Browser' or 'Login' between them.
####12:
Name: Chain Precedence[Finish Process, Review Process] | | |
Description: The 'Finish Process' activity can occur only if it is immediately preceded by 'Review Process' activity, with no other activities occurring in between.
####13:
Name: Chain Response[Buy Medicine, Recover] | | |
Description: Whenever a 'Buy Medicine' activity occurs, it must be immediately followed by a 'Recover' activity in the next event position, with no other events in between. The 'Recover' activity is allowed to happen isolated.
####14:
Name: Chain Succession[Open the Door, Close the Door] | | |
Description: Whenever a 'Open the Door' activity occurs, it must be immediately followed by a 'Close the Door' activity, with no other events in between. Conversely, whenever a 'Close the Door' event occurs, it must be immediately preceded by a 'Open the Door' event, with no other events in between.
####15:
Name: Co-Existence[Send Inquiry, Execute Demand] | | |
Description: If 'Send Inquiry' activity occurs, then 'Execute Demand' must also occur in the same process instance, and vice-versa.
####16:
Name: Precedence[Serve Meals, Deliver Speech] | | |
Description: The 'Serve Meals' activity is permitted to occur only if a 'Deliver Speech' activity has occurred before it within the same process instance.
####17:
Name: Responded Existence[Login, Logout] | | |
Description: If 'Login' activity occurs in a process instance, then 'Logout' activity must also occur at least once within the same process instance, either before or after the 'Login' event. If 'Login' does not occur, there is no requirement for 'Logout' to occur. Multiple occurrences of 'Login' are satisfied by at least one occurrence of 'Logout', and multiple 'Logout' instances are allowed.
####18:
Name: Response[Open Browser, Fill URL] | | |
Description: If a 'Open Browser' activity occurs, then a 'Fill URL' activity must eventually occur after the 'Open Browser' event at some point in the same process instance. There is no requirement on how soon or what events may occur in between. Multiple 'Open Browser' can be pending at the same time, and a single 'Fill URL' can fulfill the obligation for all preceding 'Open Browser'. If 'Open Browser' does not occur, the constraint imposes no obligation on the occurrence of 'Fill URL'.
####19:
Name: Succession[Open Umbrella, Close Umbrella] | | |
Description: A 'Open Umbrella' activity must be followed by a 'Close Umbrella' activity at some point later in the same process instance. Conversely, a 'Close Umbrella' event can occur only if an 'Open Umbrella' event has occurred before it within the same process instance. This bidirectional and ordered dependency ensures that 'Open Umbrella' impose obligations for future 'Close Umbrella', and 'Close Umbrella' cannot occur without prior 'Open Umbrella'. Multiple 'Open Umbrella' and 'Close Umbrella' are allowed, with each 'Open Umbrella' eventually leading to a 'Close Umbrella' and each 'Close Umbrella' being justified by a preceding 'Open Umbrella'.
####20:
Name: Not Chain Succession[Style Hair, Take a Shower] | | |
Description: A 'Style Hair' activity cannot be immediately followed by a 'Take a Shower' activity; there must be at least one other event between them. Similarly, a 'Take a Shower' event cannot be immediately preceded by a 'Style Hair' event. This constraint allows multiple occurrences of 'Style Hair' and 'Take a Shower' in any order, provided they are not adjacent in the event sequence.
####21:
Name: Not Co-Existence[Go to Party, Stay at Home] | | |
Description: The 'Go to Party' activity and the 'Stay at Home' activity cannot both occur within the same process instance. This means that if 'Go to Party' event occurs one or more times, 'Stay at Home' must not occur at all in that process instance, and vice-versa. It is acceptable for neither event to occur. The constraint enforces a mutual exclusion between 'Go to Party' and 'Stay at Home', allowing for different process variants based on which event (if any) occurs.
####22:
Name: Not Succession[Eat Food, Run] | | |
Description: The 'Eat Food' activity and 'Run' activity must not occur such that the 'Eat Food' event is followed by the 'Run' event at any point later in the same process instance. Similarly, a 'Run' event must not be preceded by an 'Eat Food' event. This means that once an 'Eat Food' occurs, 'Run' cannot occur afterward, and if 'Run' event occurs, it cannot have been preceded by an 'Eat Food' event. Both events can occur in the same process instance, but not in the prohibited sequence.
####23:
Name: Not Chain Precedence[Order Confirmation, Order Placement] | | |
Description: The 'Order Confirmation' activity cannot be directly preceded by an 'Order Placement' activity, meaning that a 'Order Confirmation' event cannot occur immediately after an 'Order Placement' without any intervening events.
####24:
Name: Not Chain Response[Submit Order, Confirm Order] | | |
Description: The 'Submit Order' activity cannot be directly followed by a 'Confirm Order' activity, meaning that a 'Confirm Order' event cannot occur immediately after a 'Submit Order' without any intervening events.
####25:
Name: Not Precedence[Approve Order, Process Payment] | | |
Description: A 'Process Payment' activity cannot be preceded by a 'Approve Order' within the same process instance. If a 'Approve Order' event has occurred at any point before a 'Process Payment' event, then 'Process Payment' is disallowed. Multiple instances of 'Process Payment' and 'Approve Order' can occur, but all instances of 'Process Payment' must occur before any instance of 'Approve Order'.
####26:
Name: Not Responded Existence[Submit Order, Cancel Order] | | |
Description: If a 'Submit Order' activity occur in a process instance, then a 'Cancel Order' activity must not occur anywhere in the same process instance after the 'Submit Order' event. This means that the occurrence of 'Submit Order' event prohibits any subsequent occurrence of 'Cancel Order' event. However, if 'Submit Order' does not occur, 'Cancel Order' may occur freely. Multiple occurrences of 'Submit Order' are allowed, provided 'Cancel Order' does not occur at all in the process instance.
####27:
Constraint Template: Not Response[Submit Application, Reject Application] | | |
Description: The 'Submit Application' activity must not be followed by a 'Reject Application' activity at any point later in the same process instance. This means that once a 'Submit Application' event occurs, 'Reject Application' cannot occur afterward. However, 'Reject Application' events occurring before the 'Submit Application' are permitted. The constraint ensures that after the occurrence of 'Submit Application', the process does not allow any further occurrences of 'Reject Application'.
####28:
Name: Choice[Receive Payment, Issue Refund] | | |
Description: At least one of the activities 'Receive Payment' or 'Issue Refund' must occur in the process instance. This means that the process is compliant if 'Receive Payment' event occurs, 'Issue Refund' event occurs, or both occur. There is no restriction on the order or the number of times they occur. However, if neither 'Receive Payment' nor 'Issue Refund' occurs, the constraint is violated.
####29:
Name: Exclusive Choice[Approve Application, Reject Application] | | |
Description: Exactly one of the activities 'Approve Application' or 'Reject Application' must occur in the process instance. This means that the process is compliant if only 'Approve Application' event occurs without any occurrence of 'Reject Application' event, or only 'Reject Application' occurs without any occurrence of 'Approve Application'. The process is non-compliant if both 'Approve Application' and 'Reject Application' occur, or if neither occurs. The constraint enforces a mutual exclusivity and a mandatory occurrence of one of the specified activities.
<\Assistant Prompt>"""

To structure each construct part (i.e., activities, attributes binds and constraints) description we employed function calling. This time, we separated each construct in a function call and applied a template of generation of description.

$$
\begin{array}{ll}
\textbf{Function} & \textbf{Description} \\
\hline
\text{get_activities_description} & f(\text{activities}) \rightarrow \text{description} \\
\text{get_attributes_description} & f(\text{attributes}) \rightarrow \text{description} \\
\text{get_binds_description} & f(\text{binds}) \rightarrow \text{description} \\
\text{get_constraints_description} & f(\text{constraints}) \rightarrow \text{description} \\
\hline
\end{array}
$$

$$
\begin{aligned}
\text{where} \\
f(x) &: \text{Apply template to each } x_i \in x \rightarrow \text{generated description} \\
x &\in \{\text{activities}, \text{attributes}, \text{binds}, \text{constraints}\} \\
x_i &: \{ \text{name} : \text{string}, \text{description} : \text{string} \} \\
\text{Required Properties:} \\
\text{name} &: \text{The name of the } x_i \\
\text{description} &: \text{The description of the } x_i \\
\end{aligned}
$$


In [ ]:
# 4 functions to get model description.
toolsdescription = [
  {
    "type": "function",
    "function": {
      "name": "get_activities_description",
      "description": "Fetch the activities of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "activities": {
            "type": "array",
            "description": "The activities contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the activity."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the activity."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_attributes_description",
      "description": "Fetch the attributes of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "attributes": {
            "type": "array",
            "description": "The attributes contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the attribute."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the attribute."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_binds_description",
      "description": "Fetch the binds of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "binds": {
            "type": "array",
            "description": "The binds contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the bind."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the bind."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_constraints_description",
      "description": "Fetch the constraints of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "constraints": {
            "type": "array",
            "description": "The constraints contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the constraint."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the constraint."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  }
]

In [ ]:
model ="\n".join(declare_model.declare_model_lines)
intermediary_input_prompt = "New model to generate description of activities, attributes, binds and constraints based in the template and examples: \n" + model
print(intermediary_input_prompt)

In [ ]:
intermediary_input_prompt="""New model to generate description of activities, attributes, binds and constraints based in the template and examples:
activity Request Equipment
activity Approve Request
activity Order Equipment
activity Receive Equipment
activity Inspect Equipment
activity Install Equipment
activity Train Staff
activity Use Equipment
activity Maintain Equipment
activity Decommission Equipment
bind Request Equipment: Request ID
bind Approve Request: Approval Status
bind Order Equipment: Equipment Cost
bind Inspect Equipment: Equipment Condition, Inspection Score
bind Train Staff: Training Hours
bind Install Equipment: Installation Status
bind Maintain Equipment: Maintenance Status
bind Decommission Equipment: Decommission Reason
bind Use Equipment: Usage Frequency
Request ID: integer between 1 and 10000
Equipment Cost: float between 100.0 and 100000.0
Approval Status: Pending, Approved, Rejected
Equipment Condition: New, Good, Damaged
Training Hours: integer between 1 and 40
Installation Status: Not Started, In Progress, Completed
Maintenance Status: Up to Date, Due, Overdue
Usage Frequency: integer between 0 and 100
Decommission Reason: Obsolete, Faulty, Replaced
Inspection Score: float between 0.0 and 10.0
Init[Request Equipment] | |
End[Decommission Equipment] | |
Existence[Inspect Equipment] | |
Absence[Request Equipment] | |
Precedence[Order Equipment, Approve Request] | |A.Approval Status is Approved |
Chain Response[Receive Equipment, Inspect Equipment] | | |0,2,d
Response[Install Equipment, Train Staff] |A.Installation Status is Completed | |
Precedence[Use Equipment, Train Staff] | | |
Precedence[Use Equipment, Maintain Equipment] | |T.Maintenance Status is Up to Date |
Not Chain Succession[Use Equipment, Decommission Equipment] | | |
Responded Existence[Inspect Equipment, Decommission Equipment] |A.Equipment Condition is Damaged | |
Precedence[Order Equipment, Approve Request] |A.Equipment Cost > 1000 |T.Approval Status is Approved |
Chain Response[Use Equipment, Maintain Equipment] | | |0,30,d
Precedence[Install Equipment, Inspect Equipment] | | |
Responded Existence[Decommission Equipment, Inspect Equipment] |A.Decommission Reason is Obsolete | |
"""

In [ ]:
# Run prompt to generate intermediary description
messagesmptodesc=[
            {
                "role": "system",
                "content": decltotextassistant
            },
            {
                "role": "user",
                "content": intermediary_input_prompt
            }
        ]

response_decltointermediary = chat_completion_request(
        messages=messagesmptodesc,
        tools= toolsdescription,
        tool_choice="auto",
        temperature=0
        )

In [ ]:
tool_call = response_decltointermediary.choices[0].message.tool_calls
function_arguments = tool_call

In [ ]:
def process_intermediate_descriptions(data):
    model = {
        "Activities": [],
        "Attributes": [],
        "Binds": [],
        "Constraints": []
    }

    for item in data:
        func_args = json.loads(item.function.arguments)
        for key, value in func_args.items():
            for v in value:
                model[key.capitalize()].append({
                    "name": v['name'],
                    "description": v['description']
                })

    model_string = ""
    for key, items in model.items():
        model_string += f"## {key}\n"
        for item in items:
            model_string += f"- **{item['name']}**: {item['description']}\n"
        model_string += "\n"

    return model_string, model


intermediary_description_string, intermediary_structured_model = process_intermediate_descriptions(response_decltointermediary.choices[0].message.tool_calls)

In [ ]:
# The main prompt used to generate the model reconstruction from descriptions.
decl_reconstruction_assistant="""
<Assistant Prompt>
You are an an expert in declarative business process management, specifically for interpreting and modeling Multi-Perspective Declare models (MP-Declare). Your primary task is to analyze the provided MP-Declare description and model the process in the given schema:
Use the following knowledge of MP-Declare to know it syntax and how to model:
<MP-Declare Model Specification>
#**Multi-Perspective Declare (MP-Declare) Constructs**

##**Activities**

<Activities Specification Guidelines>
Activities are specified with the word 'activity' followed by descriptive names, which should use active verbs, avoid abbreviations, and follow naming conventions (i.e., <Action Verb> <Object> or <Object> <Action Verb>).
</Activities Specification Guidelines>

**Examples of activities specification:**
<Activities Examples>
activity Driving Test
activity Initiate Purchase Order
</Activities Examples>
</Activities Specification Guidelines>

##**Binds**

<Binds Specification Guidelines>
Binds are specified as a link between activities and attributes. They have as a prefix the activity name followed by ':' and then a list of attributes, each separated by ','.

**Examples of binds specification:**
<Binds Examples>
Driving Test: Driver, Grade
Initiate Purchase Order: Purchase ID, Customer ID, Order Status, Order Value
</Binds Examples>
</Binds Specification Guidelines>

##**Attributes**
<Attributes Specification Guidelines>
MP-Declare models supports three types of attributes: integer, float and enumeration.

**Attribute Types:**
- **Integer:** Represents numeric integer values within a specified range.
- **Float:** Represents numeric values with floating-point within a specified range.
- **Enumeration:** Represents a limited set of possible values.

**Integer specification:**<Integer Attribute Specification Guidelines>[{Attribute Name}: integer between {min_value} and {max_value}], where {attribute name} is the name of attribute and {min_value} and {max_value} are two integer numbers.</Integer Attribute Specification Guideline>
**Float specification:**<Float Attribute Specification Guideline>[{Attribute Name}: float between {min_value} and {max_value}], where {attribute name} is the name of attribute and {min_value} and {max_value} are two float numbers.</Float Attribute Specification Guideline>
**Enumeration specification:**<Enumeration Specification Guideline>[{Attribute Name}: {value1}, {value2}, ..., {valuen}], where {attribute name} is the name of attribute and {value1} to {valuen} represent possible values for the attribute. These possible values should always be at least 2 and not infinite.</Enumeration Specification Guidelines>

**Examples of attributes specification:**
<Attributes Examples>
**Integer attribute example:** <Integer Attribute Example>Grade: integer between 1 and 5</Integer Attribute Example>.
**Float attribute example:** <Float Attribute Example>Order Value: float between 1.99 and 99999.9</Float Attribute Example>.
**Enumeration attribute example:** <Enumeration Attribute Example>Order Status: Approved, Not Approved, Shipped, Not Shipped</Enumeration Attribute Example>.
</Attributes Examples>
</Attributes Specification Guidelines>

##**Constraints**

MP-Declare has two types of templates: unary and binary. Following is specified the templates and their semantics.

###**Unary Templates**

Unary templates have only Activation activity.

<Unary Constraints Templates>
Constraint Template: Absence[n][Activation]
Semantics: Activation must not occur n times in the process.

Constraint Template: Exactly[n][Activation]
Semantics: Activation must occur exactly n times in the process.

Constraint Template: Existence[n][Activation]
Semantics: Activation must occur at least n times in the process.

Constraint Template: Init[Activation]
Semantics: Activation must be the first activity to occur in the process.

Constraint Template: End[Activation]
Semantics: Activation must be the last activity to occur in the process.
</Unary Constraints Templates>

###**Binary Templates**
Binary templates have both Activation and Target activities.
####**Binary Positive Templates**

<Binary Positive Templates>
Constraint Template: Alternate Precedence[Target, Activation]
Semantics: An Activation event can occur only if it was immediately preceded by a Target event, and no other Target event is allowed between that Target and the Activation. This ensures a one-to-one and ordered pairing between Targets and Activations, preventing any Activation from occurring without the necessary prior Target and avoiding overlapping or multiple Targets before an Activation.

Constraint Template: Alternate Response[Activation, Target]
Semantics: If an Activation occurs, then a Target must occur after that Activation and before any subsequent Activation occurs no other occurrence of Activation is allowed between an Activation and its corresponding Target. This ensures that each Activation is individually responded to by a Target in sequence.

Constraint Template: Alternate Succession[Activation, Target]
Semantics: Each occurrence of an Activation must eventually be followed by a Target, and each occurrence of a Target must be preceded by an Activation, with no intermediate occurrences of Activation or Target between them. This ensures a strict one-to-one and ordered pairing between Activations and Targets, preventing any overlapping or interruption by additional Activations or Targets in the sequence.

Constraint Template: Chain Precedence[Target, Activation]
Semantics: A Target event can occur only if it is immediately preceded by an Activation event, with no other events occurring in between. There can be multiple Activation-Target pairs within the process, each adhering to this immediate precedence rule. Activations may occur without being followed by a Target, but Targets cannot occur without such an immediate preceding Activation.

Constraint Template: Chain Response[Activation, Target]
Semantics: Whenever an Activation occurs, it must be immediately followed by a Target in the next event position, with no other events in between. This immediate succession ensures a direct and uninterrupted transition from Activation to Target. Multiple Activation-Target pairs can exist in a process instance, each adhering to this immediate succession rule. Events of Target can occur elsewhere in the process but not between an Activation and its corresponding Target.

Constraint Template: Chain Succession[Activation, Target]
Semantics: Whenever an Activation event occurs, it must be immediately followed by a Target event, with no other events in between. Conversely, whenever a Target event occurs, it must be immediately preceded by an Activation event, with no other events in between. This enforces a strict one-to-one and immediate pairing of Activations and Targets, ensuring that neither occurs without the other and that they occur consecutively without interruption.

Constraint Template: Co-Existence[Activation, Target]
Semantics: If Activation occurs in a process instance, then Target must also occur in the same process instance, and vice versa. This mutual obligation ensures that neither event can occur without the other. The events can occur multiple times and in any order relative to each other. If neither event occurs, the constraint is still considered satisfied.

Constraint Template: Precedence[Target, Activation]
Semantics: An Activation event is permitted to occur only if a Target event has occurred before it within the same process instance. Activations that occur without any prior Target are violations of the constraint. Multiple Targets can occur, and any number of events (including other Activations) can happen between a Target and a subsequent Activation. This ensures that certain actions cannot proceed without the necessary prerequisites being met first.

Constraint Template: Responded Existence[Activation, Target]
Semantics: If Activation occurs in a process instance, then Target must also occur at least once within the same process instance, either before or after the Activation. If Activation does not occur, there is no requirement for Target to occur. Multiple occurrences of Activation are satisfied by at least one occurrence of Target, and multiple Targets are allowed.

Constraint Template: Response[Activation, Target]
Semantics: If an Activation occurs, then a Target must eventually occur after the Activation at some point in the same process instance. There is no requirement on how soon or what events may occur in between. Multiple Activations can be pending at the same time, and a single Target can fulfill the obligation for all preceding Activations. If Activation does not occur, the constraint imposes no obligation on the occurrence of Target.

Constraint Template: Succession[Activation, Target]
Semantics: An Activation event must be followed by a Target event at some point later in the same process instance. Conversely, a Target event can occur only if an Activation event has occurred before it within the same process instance. This bidirectional and ordered dependency ensures that Activations impose obligations for future Targets, and Targets cannot occur without prior Activations. Multiple Activations and Targets are allowed, with each Activation eventually leading to a Target and each Target being justified by a preceding Activation..
</Binary Positive Templates>

####**Binary Negative Templates**

<Binary Negative Templates>
Constraint Template: Not Chain Succession[Activation, Target]
Semantics: An Activation event cannot be immediately followed by a Target event; there must be at least one other event between them. Similarly, a Target event cannot be immediately preceded by an Activation event. This constraint allows multiple occurrences of Activations and Targets in any order, provided they are not adjacent in the event sequence.

Constraint Template: Not Co-Existence[Activation, Target]
Semantics: The Activation event and the Target event cannot both occur within the same process instance. This means that if Activation occurs one or more times, Target must not occur at all in that process instance, and vice versa. It is acceptable for neither event to occur. The constraint enforces a mutual exclusion between Activation and Target, allowing for different process variants based on which event (if any) occurs.

Constraint Template: Not Succession[Activation, Target]
Semantics: An Activation event and a Target event must not occur such that the Activation is followed by the Target at any point later in the same process instance. Similarly, a Target event must not be preceded by an Activation event. This means that once an Activation occurs, Target cannot occur afterward, and if a Target occurs, it cannot have been preceded by an Activation. Both events can occur in the same process instance, but not in the prohibited sequence.

Constraint Template: Not Chain Precedence[Target, Activation]
Semantics: A Target event cannot be directly preceded by an Activation event, meaning that a Target cannot occur immediately after an Activation without any intervening events.

Constraint Template: Not Chain Response[Activation, Target]
Semantics: An Activation event cannot be directly followed by a Target event, meaning that a Target cannot occur immediately after an Activation without any intervening events.

Constraint Template: Not Precedence[Target, Activation]
Semantics: An Activation event cannot be preceded by a Target event within the same process instance. This means that if any Target event has occurred before an Activation event in the process instance, then that Activation is disallowed. Multiple Activations and Targets can occur, but all instances of the Activation must occur before any instance of the Target.

Constraint Template: Not Responded Existence[Activation, Target]
Semantics: If an Activation event occurs in a process instance, then a Target event must not occur anywhere in the same process instance after the Activation. This means that the occurrence of Activation prohibits any subsequent occurrence of Target. However, if Activation does not occur, Target may occur freely. Multiple occurrences of Activation are allowed, provided Target does not occur at all in the process instance.

Constraint Template: Not Response[Activation, Target]
Semantics: An Activation event must not be followed by a Target event at any point later in the same process instance. This means that once an Activation occurs, Target cannot occur afterward. However, Target events occurring before the Activation are permitted. The constraint ensures that after the occurrence of Activation, the process does not allow any further occurrences of Target.
</Binary Negative Templates>

####**Choice Templates**

<Choice Templates>
Constraint Template: Choice[Activation, Target]
Semantics: At least one of the activities Activation or Target must occur in the process instance. This means that the process is compliant if Activation occurs, Target occurs, or both occur. There is no restriction on the order or the number of times they occur. However, if neither Activation nor Target occurs, the constraint is violated.

Constraint Template: Exclusive Choice[Activation, Target]
Semantics: Exactly one of the activities Activation or Target must occur in the process instance. This means that the process is compliant if only Activation occurs without any occurrence of Target, or only Target occurs without any occurrence of Activation. The process is non-compliant if both Activation and Target occur, or if neither occurs. The constraint enforces a mutual exclusivity and a mandatory occurrence of one of the specified activities.
</Choice Templates>

##**Conditions**
<Conditions>
There are three types of conditions in MP-Declare: activation, correlation and time.
<Activation and Correlation>
###**Activation and Correlation:**
<Activation Condition Example>
A.Phone Number
</Activation Condition Example>
<Correlation Condition Example>
T.Phone Number
</Correlation Condition Example>

###**Conditions Operations Guidelines**
<Conditions Operations Guidelines>
Conditions have operations on attributes and should always reference existing attributes values that are bound to the activities of the condition.

<Enumeration Operations>Operations on attribute values for enumeration: is, is not, in, not in.</Enumeration Operations>

**Examples of Operations in Enumeration Attributes:**
<Enumeration Operations Examples>
A.Transport Type is Car
A.Transport Type is not Car
A.Transport Type in (Car, Train)
A.Transport Type not in (Car, Train)

If both activation condition and correlation condition have the same attribute, == can be specified for equals, and != can be specified as different in the correlation condition section.

**Example for ==:**
<Example double equals operator>A.Transport Type == T.Transport Type</Example double equals operator>
**Example for or !=:**
<Example not equal to operator>A.Transport Type != T.Transport Type</Example not equal to operator>
</Enumeration Operations Examples>

<Integer or Float Operations>Operations on attribute values for numeric (integer or float) attribute: <, <=, =, =>, >.</Integer or Float Operations>
**Examples of Operations in Float or Integer Attributes:**
<Integer or Float Operations Examples>
A.Price > 10
A.Price <= 5
A.Price == 3
</Integer or Float Operations Examples>

<Logical Operations>Operations can be joined with 'and' and 'or'.</Logical Operations>
<Logical Operations Examples>
(T.Price <= 10) OR (T.Price > 100)
(T.Price = 10) AND (T.Transport Type is Car)
</Logical Operations Examples>

</Conditions Operations Guidelines>
</Activation and Correlation>

<Time Condition Guidelines>
The time condition must be specified as follows: min_value, max_value, measure. Where min_value and max_value are positive integers and min_value<=max_value and measure refers to the measure of time which can be s for seconds, m for minutes, h for hours or d for days.
</Time Condition Guidelines>

**Examples of Time Conditions Specification:**
<Time Condition Examples>
10,15,s: means that the condition must be triggered between 10 and 15 seconds.
5,13,m: means that the condition must be triggered between 5 and 13 minutes.
0,48,h: means that the condition must be triggered in at most 48 hours.
0,2,d: means that the condition must be triggered in at most 2 days.
</Time Condition Examples>
</Conditions>

Examples of model descriptions are provided below to guide your output and ensure alignment between data representation and text description. Your output will be the name of the construct and generated description.

###Activity Examples
####1
Name: activity Plan Event
Description:The 'Plan Event' activity represents the planning phase for organizing an event.
####2
Name: activity Collect Feedback
Description: The 'Collect Feedback' activity includes gathering feedback from guests and staff about the event.
####3
Name: activity Coordinate Photography
Description: The 'Coordinate Photography' activity involves managing photography services to capture moments during the event.

###Attributes Exampels
####1
Name: User ID: Integer between 1 and 1000
Description: The attribute 'User ID: Integer between 1 and 1000' represents the user's identification number for authentication purposes.
####2
Name: User Password: Float between 1000000 and 9000000
Description: The attribute 'User Password: Float between 1000000 and 9000000' represents the user's password for login.
####3
Name: Product Type: Cloths, radio, computer, notebook, food, chocolate
Description: The attribute 'Product Type: cloths, radio, computer, notebook, food, chocolate' specifies the types of products available for purchase.

###Binds Examples
####1
Name: bind Driving Test: Driver, Grade
Description: The bind 'Driving Test: Driver, Grade' specifies that the activity 'Driving Test' requires the attributes 'Driver' and 'Grade'.
####2
Name: bind Book Venue: Event ID, Venue
Description: The bind 'Book Venue: Event ID, Venue' links the 'Book Venue' activity with the attributes 'Event ID' and 'Venue'.
####3
Name: bind Hire Staff: Event ID, Staff Role
Description: The bind 'Hire Staff: Event ID, Staff Role' links the 'Hire Staff' activity with the attributes 'Event ID' and 'Staff Role'.

###Constraints Examples
####1
Name: Init[Plan Event] | |
Description: The 'Plan Event' must be the first activity to occur in the process.
####2
Name: End[Collect Feedback] | |
Description: The 'Collect Feedback' must be the last activity to occur in the process.
####3
Name: Existence[Conduct Internal Audit] | |
Description: The 'Conduct Internal Audit' activity must occur at least once in the process.
####4
Name: Existence4[Declare Contigency] | |
Description: The 'Declare Contigency' activity must occur at least 4 times in the process.
####5
Name: Absence[Stop Process] | |
Description: The 'Stop Process' must not occur in the process.
####6
Name: Absence3[Retry Access] | |
Description: The 'Retry Access' must not occur 3 times in the process.
####7:
Name: Exactly[Send Message] | |
Description: The 'Send Message' activity must occur exactly once in the process.
####8:
Name: Exactly5[Get Medicines] | |
Description: The 'Get Medicines' activity must occur exactly 5 times in the process.
####9:
Name: Alternate Precedence[Pay Bill, Get Goods] | | |
Description: The 'Get Goods' activity can occur only if it was immediately preceded by a 'Pay Bill' activity, and no other 'Pay Bill' event is allowed between the 'Pay Bill' and the 'Get Goods'.
####10:
Name: Alternate Response[Walk in the Park, Drink Water]
Description: If 'Walk in the Park' activity occurs, then 'Drink Water' activity  must occur after that 'Walk in the Park' event and before any subsequent 'Walk in the Park' occurs no other occurrence of 'Walk in the Park'is allowed between an 'Walk in the Park' and 'Drink Water'.
####11:
Name: Alternate Succession[Open Browser, Login] | | |
Description: Each occurrence of a 'Open Browser' activity must eventually be followed by a 'Login' activity, and each occurrence of a 'Login' must be preceded by 'Open Browser', with no intermediate occurrences of 'Open Browser' or 'Login' between them.
####12:
Name: Chain Precedence[Finish Process, Review Process] | | |
Description: The 'Finish Process' activity can occur only if it is immediately preceded by 'Review Process' activity, with no other activities occurring in between.
####13:
Name: Chain Response[Buy Medicine, Recover] | | |
Description: Whenever a 'Buy Medicine' activity occurs, it must be immediately followed by a 'Recover' activity in the next event position, with no other events in between. The 'Recover' activity is allowed to happen isolated.
####14:
Name: Chain Succession[Open the Door, Close the Door] | | |
Description: Whenever a 'Open the Door' activity occurs, it must be immediately followed by a 'Close the Door' activity, with no other events in between. Conversely, whenever a 'Close the Door' event occurs, it must be immediately preceded by a 'Open the Door' event, with no other events in between.
####15:
Name: Co-Existence[Send Inquiry, Execute Demand] | | |
Description: If 'Send Inquiry' activity occurs, then 'Execute Demand' must also occur in the same process instance, and vice-versa.
####16:
Name: Precedence[Serve Meals, Deliver Speech] | | |
Description: The 'Serve Meals' activity is permitted to occur only if a 'Deliver Speech' activity has occurred before it within the same process instance.
####17:
Name: Responded Existence[Login, Logout] | | |
Description: If 'Login' activity occurs in a process instance, then 'Logout' activity must also occur at least once within the same process instance, either before or after the 'Login' event. If 'Login' does not occur, there is no requirement for 'Logout' to occur. Multiple occurrences of 'Login' are satisfied by at least one occurrence of 'Logout', and multiple 'Logout' instances are allowed.
####18:
Name: Response[Open Browser, Fill URL] | | |
Description: If a 'Open Browser' activity occurs, then a 'Fill URL' activity must eventually occur after the 'Open Browser' event at some point in the same process instance. There is no requirement on how soon or what events may occur in between. Multiple 'Open Browser' can be pending at the same time, and a single 'Fill URL' can fulfill the obligation for all preceding 'Open Browser'. If 'Open Browser' does not occur, the constraint imposes no obligation on the occurrence of 'Fill URL'.
####19:
Name: Succession[Open Umbrella, Close Umbrella] | | |
Description: A 'Open Umbrella' activity must be followed by a 'Close Umbrella' activity at some point later in the same process instance. Conversely, a 'Close Umbrella' event can occur only if an 'Open Umbrella' event has occurred before it within the same process instance. This bidirectional and ordered dependency ensures that 'Open Umbrella' impose obligations for future 'Close Umbrella', and 'Close Umbrella' cannot occur without prior 'Open Umbrella'. Multiple 'Open Umbrella' and 'Close Umbrella' are allowed, with each 'Open Umbrella' eventually leading to a 'Close Umbrella' and each 'Close Umbrella' being justified by a preceding 'Open Umbrella'.
####20:
Name: Not Chain Succession[Style Hair, Take a Shower] | | |
Description: A 'Style Hair' activity cannot be immediately followed by a 'Take a Shower' activity; there must be at least one other event between them. Similarly, a 'Take a Shower' event cannot be immediately preceded by a 'Style Hair' event. This constraint allows multiple occurrences of 'Style Hair' and 'Take a Shower' in any order, provided they are not adjacent in the event sequence.
####21:
Name: Not Co-Existence[Go to Party, Stay at Home] | | |
Description: The 'Go to Party' activity and the 'Stay at Home' activity cannot both occur within the same process instance. This means that if 'Go to Party' event occurs one or more times, 'Stay at Home' must not occur at all in that process instance, and vice-versa. It is acceptable for neither event to occur. The constraint enforces a mutual exclusion between 'Go to Party' and 'Stay at Home', allowing for different process variants based on which event (if any) occurs.
####22:
Name: Not Succession[Eat Food, Run] | | |
Description: The 'Eat Food' activity and 'Run' activity must not occur such that the 'Eat Food' event is followed by the 'Run' event at any point later in the same process instance. Similarly, a 'Run' event must not be preceded by an 'Eat Food' event. This means that once an 'Eat Food' occurs, 'Run' cannot occur afterward, and if 'Run' event occurs, it cannot have been preceded by an 'Eat Food' event. Both events can occur in the same process instance, but not in the prohibited sequence.
####23:
Name: Not Chain Precedence[Order Confirmation, Order Placement] | | |
Description: The 'Order Confirmation' activity cannot be directly preceded by an 'Order Placement' activity, meaning that a 'Order Confirmation' event cannot occur immediately after an 'Order Placement' without any intervening events.
####24:
Name: Not Chain Response[Submit Order, Confirm Order] | | |
Description: The 'Submit Order' activity cannot be directly followed by a 'Confirm Order' activity, meaning that a 'Confirm Order' event cannot occur immediately after a 'Submit Order' without any intervening events.
####25:
Name: Not Precedence[Approve Order, Process Payment] | | |
Description: A 'Process Payment' activity cannot be preceded by a 'Approve Order' within the same process instance. If a 'Approve Order' event has occurred at any point before a 'Process Payment' event, then 'Process Payment' is disallowed. Multiple instances of 'Process Payment' and 'Approve Order' can occur, but all instances of 'Process Payment' must occur before any instance of 'Approve Order'.
####26:
Name: Not Responded Existence[Submit Order, Cancel Order] | | |
Description: If a 'Submit Order' activity occur in a process instance, then a 'Cancel Order' activity must not occur anywhere in the same process instance after the 'Submit Order' event. This means that the occurrence of 'Submit Order' event prohibits any subsequent occurrence of 'Cancel Order' event. However, if 'Submit Order' does not occur, 'Cancel Order' may occur freely. Multiple occurrences of 'Submit Order' are allowed, provided 'Cancel Order' does not occur at all in the process instance.
####27:
Constraint Template: Not Response[Submit Application, Reject Application] | | |
Description: The 'Submit Application' activity must not be followed by a 'Reject Application' activity at any point later in the same process instance. This means that once a 'Submit Application' event occurs, 'Reject Application' cannot occur afterward. However, 'Reject Application' events occurring before the 'Submit Application' are permitted. The constraint ensures that after the occurrence of 'Submit Application', the process does not allow any further occurrences of 'Reject Application'.
####28:
Name: Choice[Receive Payment, Issue Refund] | | |
Description: At least one of the activities 'Receive Payment' or 'Issue Refund' must occur in the process instance. This means that the process is compliant if 'Receive Payment' event occurs, 'Issue Refund' event occurs, or both occur. There is no restriction on the order or the number of times they occur. However, if neither 'Receive Payment' nor 'Issue Refund' occurs, the constraint is violated.
####29:
Name: Exclusive Choice[Approve Application, Reject Application] | | |
Description: Exactly one of the activities 'Approve Application' or 'Reject Application' must occur in the process instance. This means that the process is compliant if only 'Approve Application' event occurs without any occurrence of 'Reject Application' event, or only 'Reject Application' occurs without any occurrence of 'Approve Application'. The process is non-compliant if both 'Approve Application' and 'Reject Application' occur, or if neither occurs. The constraint enforces a mutual exclusivity and a mandatory occurrence of one of the specified activities.
<\Assistant Prompt>"""

In [ ]:
model_reconstructred = structured_output(
    messages = [
        {
            "role": "system",
            "content": decl_reconstruction_assistant
        },
        {
            "role": "user",
            "content": f"Create a MP-Declare model of the following process using the knowledge from the assistant: \n {intermediary_description_string}",
        }
    ],
    response_format=MPDeclareModel,
    temperature=0
)

#Save reconstructed model JSON and DECL files.
json_filename = 'reconstructed.json'
decl_filename = 'reconstructed.decl'
with open(json_filename, 'w') as json_file:
        json.dump(model_reconstructred.choices[0].message.parsed.model_dump_json(), json_file, indent=4)

with open(decl_filename, 'w') as decl_file:
        decl_file.write(model_reconstructred.choices[0].message.parsed.convert_to_string())


## Final Description (Agentic Flows)

This is the final step of our solution. It will take as input the intermediary descriptions and outputs the complete description of the MP-Declare model. It is necessary to mention that this entire description entangles all aspects of model constructs and how they interplay with each other.
In order to generate the description, the model will take a prompt with instructs in format of role-play and the intermediary descriptions of the process.

In [ ]:
guidelines="""
#Trace-Related:
    - Use Background for Init constraints to establish initial context.
    - Apply Conclusion for End constraint to close the model coherently.
    #Frequency:
    - Use Elaboration for Existence constraints to add depth and context regarding the presence of activities within the process.
    - Use Contrast for Absence constraints to highlight exclusions and reinforce process integrity by specifying what is not permissible.
    - Apply Summary for Exactly constraints to set fixed boundaries on activity occurrences, ensuring precision within the process model.
    #Dependency:
    - Use Cause-Effect for Responded Existence constraints to establish dependency, where one activity's occurrence necessitates the presence of another, enhancing coherence in the process.
    #Flexible Ordering:
    - Use Background for Precedence constraints to set up an initial context where one activity must occur before another, guiding the logical sequence within the model.
    - Apply Sequence for Response constraints to define an ordered flow where one activity follows another, ensuring a cohesive temporal structure in the model.
    - Use Cause-Effect for Succession constraints to establish a causal link, ensuring each activity follows logically from the previous one, creating a cohesive progression within the model.
    #Alternate Ordering:
    - Apply Condition-Consequence for Alternate Precedence constraints to ensure order consistency, requiring each occurrence of a target activity to be preceded by a specified activity without overlap, promoting balanced process flow.
    - Use Condition-Consequence for Alternate Response constraints to promote balance by ensuring each occurrence of an activation is followed by a unique target, with no intervening activations, supporting clear one-to-one activity pairing.
    - Use Condition-Consequence for Alternate Succession constraints to enforce an ordered structure with unique activity pairings, ensuring each activation is followed by a target and each target is preceded by an activation, without intervening occurrences, enhancing process coherence.
    #Immediate Ordering:
    - Use Sequence for Chain Precedence constraints to establish immediate dependency, ensuring that each target follows an activation directly, with no intervening events, making the sequence explicit and precise.
    - Apply Cause-Effect for Chain Response constraints to capture immediate succession, ensuring a direct cause-and-effect relationship where each activation is immediately followed by its target.
    - Apply Cause-Effect for Chain Succession constraints to enforce a tightly coupled sequence, ensuring each activation is directly succeeded by its target with no intervening activities.
    #Mutual Dependency:
    - Use Conjunction for Co-Existence constraints to establish mutual dependency, ensuring that both activities occur together within the process flow.
    #Exclusivity:
    - Use Contrast for Choice constraints to introduce alternative paths, allowing flexibility by permitting either activity to occur within the process.
    - Use Alternative for Exclusive Choice constraints to enforce mutually exclusive pathways, ensuring that exactly one of the specified activities occurs within the process.
    - Apply Contrast for Not Co-Existence constraints to set exclusion rules, ensuring that specified activities do not occur together within the same process instance, thereby maintaining process integrity by preventing conflicting events.
    #Negative Dependency:
    - Apply Contrast for Not Responded Existence constraints to prevent certain dependencies, ensuring that specified activities do not occur in response to one another, thus refining the model by disallowing specific contingent occurrences.
    #Negative Flexible Ordering:
    - Apply Contrast for Not Precedence constraints to restrict activities from occurring in a specific order, ensuring that designated activities do not precede others, thereby preserving the intended sequence within the model.
    - Apply Contrast for Not Response constraints to prevent activities from following a specific sequence, ensuring that designated activities do not occur after certain others, thus maintaining the integrity of the model's intended progression.
    - Apply Contrast for Not Succession constraints to restrict overlapping sequences, ensuring that certain activities do not follow others at any point, preserving the model's defined flow.
    #Negative Immediate Ordering:
    - Apply Contrast for Not Chain Precedence constraints to enforce non-immediate order restrictions, ensuring that specified activities are not immediately preceded by others, maintaining separation within the sequence.
    - Apply Contrast for Not Chain Response constraints to prevent immediate succession, ensuring that specified activities do not immediately follow one another, preserving structural integrity in the sequence.
    - Use Contrast for Not Chain Succession constraints to prevent direct succession, ensuring that specified activities do not immediately precede or follow each other, thereby refining the process model by excluding specific immediate sequences."""

response_template="""
The MP-Declare process model for the property transaction process is described as follows:

### **Trace-Related Constraints**
#### **Init Constraints**
The process begins with the activity 'Draft Contract,' which involves creating the initial version of the contract with 'Contract Version' and 'Contract Status' attributes.
#### **End Constraints**
The process concludes with the activity 'Close Deal,' which involves finalizing the property transaction and completing all necessary paperwork. This activity must be the last to occur in the process and must be completed within 30 days.

### **Frequency Constraints**

#### **Existence Constraint**
The 'Review Contract' activity must occur at least once in the process. This ensures that the contract is examined for accuracy and completeness, adding depth and reliability to the process.

### **Flexible Ordering Constraints**

#### **Precedence Constraints**
1. The activity 'Approve Contract' is permitted to occur only if a 'Conduct Title Search' activity has occurred before it. The 'Conduct Title Search' activity must verify that the 'Contract Status is Reviewed' and the 'Title Search Result is Clear,' ensuring that the contract is thoroughly reviewed and the property ownership is legally verified before approval.
2. The activity 'Negotiate Terms' is permitted to occur only if an 'Assess Property Value' activity has occurred before it, with the condition that the 'Property Value' is greater than 50,000. This ensures that the negotiation of terms is based on a valid and substantial property valuation.
3. The activity 'Close Deal' is permitted to occur only if a 'Verify Funds' activity has occurred before it. The 'Verify Funds' activity must confirm that the 'Deal Status is Open' and that the 'Funds Verified' are greater than or equal to the 'Property Value,' ensuring financial readiness before finalizing the deal.

#### **Response Constraints**
1. If the 'Draft Contract' activity occurs with the condition 'Contract Status is Draft,' then the 'Review Contract' activity must occur within 2 days. This ensures a timely review of the drafted contract, maintaining the process's momentum.
2. If the 'Sign Contract' activity occurs with the condition 'Contract Status is Signed,' then the 'Register Property' activity must occur with the condition 'Registration Status is Pending' within 5 days. This ensures that the property registration follows promptly after the contract is signed, maintaining the legal and procedural flow.

#### **Succession Constraints**
Whenever an 'Approve Contract' activity occurs with the condition 'Contract Status is Approved,' it must be immediately followed by a 'Sign Contract' activity with the condition 'Contract Status == Contract Status' within 1 day. This establishes a direct causal link between the approval and signing of the contract, ensuring a seamless progression in the process.

### **Exclusivity Constraints**

#### **Not Co-Existence Constraints**
The 'Sign Contract' activity and the 'Conduct Title Search' activity cannot both occur within the same process instance if the conditions 'Contract Status is Approved' and 'Title Search Result is Issues Found' are met. This exclusion ensures that a contract cannot be signed if the title search reveals issues, thereby maintaining the integrity of the process by preventing conflicting events.

### **Immediate Ordering Constraints**

#### **Chain Succession Constraints**
Whenever an 'Approve Contract' activity occurs with the condition 'Contract Status is Approved,' it must be immediately followed by a 'Sign Contract' activity with the condition 'Contract Status == Contract Status' within 1 day. This tightly coupled sequence ensures that the approval and signing of the contract occur in immediate succession, leaving no room for delays or intervening activities.
"""

In [ ]:
narrative_llm = LLM(
    model=GPT_MODEL,
    temperature=0,
    seed=42,
    verbose=True
)
narrative_agent = Agent(
    role="Narrative Generator",
    goal = (
    "Generate an interleaved narrative of given process model's constraints."
),
    backstory=(
        "You are an expert in narrative generation, specializing in creating cohesive descriptions of MP-Declare process models's constraints using rhetorical relations from the Rethorical Structure Theory."
    ),
    verbose=True,
    memory=True,
    llm=narrative_llm,
)
# Define the tasks for the Narrative Agent
CreateNarrative = Task(
    description=(
        "The MP-Declare model you will use to generate the narrative: \n" + intermediary_description_string + "\n"
        "Use the following rhetorical relations for the given group of MP-Declare constraints: \n" + guidelines
    ),
    expected_output="The interleaved narrative of the given MP-Declare process model's constraints. Strictly follow the following response template: \n" + response_template,
    agent=narrative_agent
)
narrative_process = Process.sequential


In [ ]:
crew = Crew(
    agents=[narrative_agent],
    tasks = [CreateNarrative],
    process=narrative_process,
    verbose=True
)
inputs = {'MP-Declare Model': intermediary_structured_model}
narrative_agent_result = crew.kickoff(inputs)

##Evaluation

###LLM-as-Judge

In [ ]:
nisaba_description = f"#MP-Declare Model\n{intermediary_description_string}\n## Interleaved Description\n{narrative_agent_result.raw}"

In [ ]:
print(nisaba_description)

In [ ]:
JUDGE_LLM = LLM(
    model=JUDGE_MODEL,
    seed=42,
    verbose=True
)

In [ ]:
DESCRIPTION_GUIDELINES="""
The model must have 1 main section (MP-Declare Model) that subdivides in 5 subsections: Activties, Attributes, Binds, Constraints and Interleaved Description. Each subsection must have the following guidelines:
Activities: must have the activity name and a description of actiivty.
Attributes: must have the attribute name and a description of attribute.
Binds: must have the bind name and a description of bind.
Constraints: must have the constraint name and a description of constraint.
Interleaved Description: must have the description of the constraints following order and usage of rethorical relations:
 #Trace-Related:
    - Use Background for Init constraints to establish initial context.
    - Apply Conclusion for End constraint to close the model coherently.
    #Frequency:
    - Use Elaboration for Existence constraints to add depth and context regarding the presence of activities within the process.
    - Use Contrast for Absence constraints to highlight exclusions and reinforce process integrity by specifying what is not permissible.
    - Apply Summary for Exactly constraints to set fixed boundaries on activity occurrences, ensuring precision within the process model.
    #Dependency:
    - Use Cause-Effect for Responded Existence constraints to establish dependency, where one activity's occurrence necessitates the presence of another, enhancing coherence in the process.
    #Flexible Ordering:
    - Use Background for Precedence constraints to set up an initial context where one activity must occur before another, guiding the logical sequence within the model.
    - Apply Sequence for Response constraints to define an ordered flow where one activity follows another, ensuring a cohesive temporal structure in the model.
    - Use Cause-Effect for Succession constraints to establish a causal link, ensuring each activity follows logically from the previous one, creating a cohesive progression within the model.
    #Alternate Ordering:
    - Apply Condition-Consequence for Alternate Precedence constraints to ensure order consistency, requiring each occurrence of a target activity to be preceded by a specified activity without overlap, promoting balanced process flow.
    - Use Condition-Consequence for Alternate Response constraints to promote balance by ensuring each occurrence of an activation is followed by a unique target, with no intervening activations, supporting clear one-to-one activity pairing.
    - Use Condition-Consequence for Alternate Succession constraints to enforce an ordered structure with unique activity pairings, ensuring each activation is followed by a target and each target is preceded by an activation, without intervening occurrences, enhancing process coherence.
    #Immediate Ordering:
    - Use Sequence for Chain Precedence constraints to establish immediate dependency, ensuring that each target follows an activation directly, with no intervening events, making the sequence explicit and precise.
    - Apply Cause-Effect for Chain Response constraints to capture immediate succession, ensuring a direct cause-and-effect relationship where each activation is immediately followed by its target.
    - Apply Cause-Effect for Chain Succession constraints to enforce a tightly coupled sequence, ensuring each activation is directly succeeded by its target with no intervening activities.
    #Mutual Dependency:
    - Use Conjunction for Co-Existence constraints to establish mutual dependency, ensuring that both activities occur together within the process flow.
    #Exclusivity:
    - Use Contrast for Choice constraints to introduce alternative paths, allowing flexibility by permitting either activity to occur within the process.
    - Use Alternative for Exclusive Choice constraints to enforce mutually exclusive pathways, ensuring that exactly one of the specified activities occurs within the process.
    - Apply Contrast for Not Co-Existence constraints to set exclusion rules, ensuring that specified activities do not occur together within the same process instance, thereby maintaining process integrity by preventing conflicting events.
    #Negative Dependency:
    - Apply Contrast for Not Responded Existence constraints to prevent certain dependencies, ensuring that specified activities do not occur in response to one another, thus refining the model by disallowing specific contingent occurrences.
    #Negative Flexible Ordering:
    - Apply Contrast for Not Precedence constraints to restrict activities from occurring in a specific order, ensuring that designated activities do not precede others, thereby preserving the intended sequence within the model.
    - Apply Contrast for Not Response constraints to prevent activities from following a specific sequence, ensuring that designated activities do not occur after certain others, thus maintaining the integrity of the model's intended progression.
    - Apply Contrast for Not Succession constraints to restrict overlapping sequences, ensuring that certain activities do not follow others at any point, preserving the model's defined flow.
    #Negative Immediate Ordering:
    - Apply Contrast for Not Chain Precedence constraints to enforce non-immediate order restrictions, ensuring that specified activities are not immediately preceded by others, maintaining separation within the sequence.
    - Apply Contrast for Not Chain Response constraints to prevent immediate succession, ensuring that specified activities do not immediately follow one another, preserving structural integrity in the sequence.
    - Use Contrast for Not Chain Succession constraints to prevent direct succession, ensuring that specified activities do not immediately precede or follow each other, thereby refining the process model by excluding specific immediate sequences.
"""

####Scoring Judge

In [ ]:
scoring_judge_agent = Agent(
    role="Scoring Judge",
    goal = ("Evaluate if a given Multi-Perspective Declare (MP-Declare) model description follows given guidelines."),
    backstory=(
        "You are an expert judge tasked with evaluating process descriptions generated from Multi-Perspective Declare (MP-DeclareModel) models. Your role is to provide an impartial and thorough assessment of the description's quality and effectiveness."
    ),
    verbose=True,
    memory=True,
    llm=JUDGE_LLM
)

In [ ]:
ScoreModel = Task(
    description=(f"""Provide an impartial and thorough assessment of the description's quality and effectiveness. Please carefully analyze the given MP-Declare model and final description and answer the following questions:

    MP-Declare Process Model Description:
    {nisaba_description}

    Based in the following guidelines:
    {DESCRIPTION_GUIDELINES}

    Rate from 1 to 10:

   1. How well does the description adhere to the prescribed format and organization specified in the guidelines?

   2. How effectively does the description integrate rhetorical relations to enhance the sections of the description?

   3. How accurately does the description capture and represent all model elements from the MP-Declare specification?

   4. How well does the description balance providing detailed information with maintaining overall clarity?

   5. How the description's effectiveness in explaining how data attributes and resources are integrated into the process model.

    For each question justify your answer. Provide rationale and examples from the texts to support your decision."""
    ),
    expected_output="An extensive evaluation of the given MP-Declare Model description in the light of guidelines and the questions (1 to 5), explaining the rationale, strengths and opportunities for improvement in the MP-Declare Model Description.",
    agent=scoring_judge_agent,
)

In [ ]:
scoring_judge_process = Process.sequential

crew = Crew(
    agents=[scoring_judge_agent],
    tasks = [ScoreModel],
    process=scoring_judge_process,
    verbose=True
)
inputs = {'MP-Declare Model Description': nisaba_description, 'Guidelines': DESCRIPTION_GUIDELINES}
score_process_result = crew.kickoff(inputs)

In [ ]:
model="""
activity Conduct Safety Training
activity Inspect Safety Equipment
activity Report Safety Incident
activity Evaluate Safety Compliance
activity Update Safety Policies
activity Perform Risk Assessment
activity Implement Safety Measures
activity Conduct Emergency Drill
activity Review Safety Reports
activity Approve Safety Budget
activity Monitor Safety Performance
activity Investigate Safety Breach
activity Communicate Safety Updates
activity Schedule Safety Audit
activity Certify Safety Personnel
bind Conduct Safety Training: Training Participants
bind Inspect Safety Equipment: Safety Equipment Status
bind Report Safety Incident: Incident Severity
bind Evaluate Safety Compliance: Compliance Status
bind Update Safety Policies:
bind Perform Risk Assessment: Risk Level
bind Implement Safety Measures:
bind Conduct Emergency Drill: Drill Type
bind Review Safety Reports:
bind Approve Safety Budget: Budget Allocation
bind Monitor Safety Performance:
bind Investigate Safety Breach: Breach Count
bind Communicate Safety Updates:
bind Schedule Safety Audit: Audit Frequency
bind Certify Safety Personnel: Certification Status
Incident Severity: integer between 1 and 5
Budget Allocation: float between 1000.0 and 50000.0
Compliance Status: Compliant, Non-Compliant
Drill Type: Fire, Earthquake, Evacuation
Training Participants: integer between 5 and 100
Safety Equipment Status: Operational, Needs Repair
Risk Level: float between 0.1 and 10.0
Audit Frequency: Monthly, Quarterly, Annually
Breach Count: integer between 0 and 10
Certification Status: Certified, Not Certified
Existence[Conduct Safety Training] | |
Exactly2[Inspect Safety Equipment] | |
Response[Report Safety Incident, Investigate Safety Breach] | | |0,2,d
Co-Existence[Conduct Emergency Drill, Review Safety Reports] | | |
Precedence[Implement Safety Measures, Approve Safety Budget] | | |
Exclusive Choice[Conduct Safety Training, Certify Safety Personnel] | | |
Response[Perform Risk Assessment, Implement Safety Measures] | | |0,48,h
End[Update Safety Policies] | |
Succession[Monitor Safety Performance, Review Safety Reports] | | |
Chain Response[Schedule Safety Audit, Evaluate Safety Compliance] | | |
Not Chain Succession[Communicate Safety Updates, Conduct Safety Training] | | |
Responded Existence[Report Safety Incident, Investigate Safety Breach] |A.Incident Severity > 3 | |
Choice[Conduct Emergency Drill, Monitor Safety Performance] | | |
Responded Existence[Inspect Safety Equipment, Inspect Safety Equipment] |A.Safety Equipment Status is Needs Repair | |
Response[Schedule Safety Audit, Schedule Safety Audit] |A.Audit Frequency is Monthly | |0,30,d
"""

####Pairwise Judge

In [ ]:
#Generate Zero-shot description
messageszeroshot=[
            {
                "role": "user",
                "content":"Generate a natural language description of the following Multi-Perspective Declare model:\n" + model
            }
        ]

response_zeroshotdescription = chat_completion_request(
        messages=messageszeroshot,
        temperature=0
        )
zeroshot_description=response_zeroshotdescription.choices[0].message.content

In [ ]:
pairwise_judge_agent = Agent(
    role="Pairwise Judge",
    goal = ("Evaluate which description better supports understandability and completeness of the process model."
),
    backstory=(
        "You are an expert judge evaluating the understandability and completeness of two descriptions of a declarative multi-perspective Declare (MP-Declare) process model."
    ),
    verbose=True,
    memory=True,
    llm=JUDGE_LLM
)

In [ ]:
# Define the tasks for the Narrative Agent
ScoreUnderstandability= Task(
    description=(f"""Provide an impartial and thorough assessment of the descriptiosns. Please carefully analyze both text A and B and answer the following questions:
Text A:
{zeroshot_description}

Text B:
{nisaba_description}

Please evaluate both texts on these criteria:

1. Formal Notation Enhances Precision
   - Which text better represents constraints using precise and unambiguous notation?
   - Which description more effectively translates formal MP-Declare constructs into clear textual representations?
   - Which approach better maintains the semantic precision in its descriptions?

2. Structured Presentation of Model Components
   - Which text organizes information into sections separating model parts?
   - Which approach provides distinct and well-explained categories that enhance deeper comprehension?
   - Which description better supports focused understanding of individual model parts?

3. Detailed Explanations of Constraints
   - Which text provides detailed explanations for each constraint type category?
   - Which approach more detailed explains the purpose and implications of constraints?
   - Which description better supports understanding of how and why attributes influences the process?

4. Alignment with Modeling Standards
   - Which text adheres to modeling standards to ensure consistency with established modeling practices?
   - Which approach facilitates precise technical communication among professionals of the domain of the description?
   - Which description better reflects MP-Declare's multi-perspective modeling aspects?

5. Support for Advanced Analysis
   - Which text is more suitable for professionals to understand multiple nuances of the process and do advanced analysis?
   - Which approach minimizes ambiguity, aiding in accurate analysis of complex processes?
   - Which description better supports advanced analytical needs through precision and formalism?

For each criterion, determine which text (A or B) better satisfies it. Provide rationale and examples from the texts to support your decision."""
    ),
    expected_output="A detailed evaluation of the MP-Declare Model descriptions based on five criteria, explaining the rationale, strengths, and opportunities for improvement for each description, indicating which is better for each criterion and overall.",
    agent=pairwise_judge_agent
)

In [ ]:
understandability_judge_process = Process.sequential

crew = Crew(
    agents=[pairwise_judge_agent],
    tasks = [ScoreUnderstandability],
    process=understandability_judge_process,
    verbose=True
)
inputs = {'Text A': zeroshot_description, 'Text B': nisaba_description}
pairwise_process_result = crew.kickoff(inputs)

In [ ]:
#Save descriptions and judgements
with open("nisaba_description.txt", "w") as file:
    file.write(nisaba_description)

with open("score_judge.txt", "w") as file:
    file.write(score_process_result.raw)

with open("zero-shot_description.txt", "w") as file:
    file.write(zeroshot_description)

with open("pairwise_judge.txt", "w") as file:
    file.write(pairwise_process_result.raw)

In [ ]:
import zipfile
from google.colab import files

In [ ]:
# File names
file_names = [
    "nisaba_description.txt",
    "score_judge.txt",
    "zero-shot_description.txt",
    "pairwise_judge.txt"
]

# Zip file name
zip_file_name = "descriptions_and_judgements.zip"

# Create the zip file
with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for file_name in file_names:
        zipf.write(file_name)

print(f"{zip_file_name} created successfully.")

# Download the zip file
files.download(zip_file_name)

In [ ]:
# File names to delete
file_names = [
    "nisaba_description.txt",
    "score_judge.txt",
    "zero-shot_description.txt",
    "pairwise_judge.txt",
    "descriptions_and_judgements.zip"  # Include the zip file as well
]

# Delete the files
for file_name in file_names:
    if os.path.exists(file_name):
        os.remove(file_name)
        print(f"{file_name} deleted successfully.")
    else:
        print(f"{file_name} does not exist. Skipping.")

print("All specified files have been deleted.")

###Metrics

####Complexity and Semantic Similarity

In [ ]:
!unzip "/content/Nisaba evaluation.zip" -d /content/


In [ ]:
def calculate_size_metric(data):
    """
    Calculate the size metric for a process model.

    The size metric is defined as the sum of the number of activities and constraints in the model.

    Args:
        data (dict): A dictionary representing the process model data.

    Returns:
        int: The size metric, which is the sum of activities and constraints.
    """
    num_activities = len(data.get('activities', []))
    num_constraints = len(data.get('constraints', []))
    return num_activities + num_constraints

def calculate_density_metric(data):
    """
    Calculate the density metric for a process model.

    The density metric is defined as the ratio of constraints to activities within each connected component
    of the model graph. The function returns the maximum density across all components.

    Args:
        data (dict): A dictionary representing the process model data.

    Returns:
        float: The maximum density of any connected component in the model graph.
    """
    G = nx.Graph()

    # Add activities as nodes
    activities = data.get('activities', [])
    for activity in activities:
        G.add_node(activity['name'])

    # Add constraints as edges between activities
    constraints = data.get('constraints', [])
    for constraint in constraints:
        activation = constraint.get('activation')
        if isinstance(activation, dict):
            activation_name = activation.get('name')
        else:
            activation_name = None

        target = constraint.get('target')
        if isinstance(target, dict):
            target_name = target.get('name')
        else:
            target_name = None

        if activation_name and target_name:
            # Binary constraint
            G.add_edge(activation_name, target_name)
        elif activation_name:
            # Unary constraint
            # Optionally, add a self-loop to represent unary constraints
            G.add_edge(activation_name, activation_name)

    # Calculate the density for each connected component
    densities = []
    for component in nx.connected_components(G):
        subgraph = G.subgraph(component)
        num_constraints = subgraph.number_of_edges()
        num_activities = subgraph.number_of_nodes()
        if num_activities > 0:
            density = num_constraints / num_activities
            densities.append(density)

    # Return the maximum density across all components
    return max(densities) if densities else 0

def calculate_separability_metric(data):
    """
    Calculate the separability metric for a process model.

    The separability metric is defined as the ratio of the number of connected components to the total
    number of elements (activities + constraints) in the model.

    Args:
        data (dict): A dictionary representing the process model data.

    Returns:
        float: The separability metric, which is the ratio of connected components to total elements.
    """
    G = nx.Graph()

    # Add activities as nodes
    activities = data.get('activities', [])
    for activity in activities:
        G.add_node(activity['name'])

    # Add constraints as edges between activities
    constraints = data.get('constraints', [])
    for constraint in constraints:
        activation = constraint.get('activation')
        if isinstance(activation, dict):
            activation_name = activation.get('name')
        else:
            activation_name = None

        target = constraint.get('target')
        if isinstance(target, dict):
            target_name = target.get('name')
        else:
            target_name = None

        if activation_name and target_name:
            G.add_edge(activation_name, target_name)
        elif activation_name:
            G.add_node(activation_name)  # Ensure the node is in the graph

    # Calculate the number of connected components
    num_components = nx.number_connected_components(G)
    total_elements = len(activities) + len(constraints)

    if total_elements > 0:
        return num_components / total_elements
    else:
        return 0

def calculate_constraint_variability_metric(data):
    """
    Calculate the constraint variability metric for a process model.

    The constraint variability metric is defined as the maximum entropy of constraint types
    across the connected components of the model graph.

    Args:
        data (dict): A dictionary representing the process model data.

    Returns:
        float: The maximum entropy of constraint types across connected components, representing constraint variability.
    """
    G = nx.Graph()
    activities = data.get('activities', [])
    for activity in activities:
        G.add_node(activity['name'])

    constraints = data.get('constraints', [])
    for constraint in constraints:
        activation = constraint.get('activation')
        if isinstance(activation, dict):
            activation_name = activation.get('name')
        else:
            activation_name = None

        target = constraint.get('target')
        if isinstance(target, dict):
            target_name = target.get('name')
        else:
            target_name = None

        constraint_type = constraint.get('template')

        if activation_name and target_name:
            # Binary constraint
            G.add_edge(activation_name, target_name, type=constraint_type)
        elif activation_name:
            # Unary constraint
            G.add_edge(activation_name, activation_name, type=constraint_type)
        # If neither activation nor target has a name, skip this constraint

    # Get all constraint types
    all_constraint_types = set(nx.get_edge_attributes(G, 'type').values())
    num_constraint_types = len(all_constraint_types)

    if num_constraint_types == 0:
        return 0  # No constraints, so variability is 0

    max_entropy = 0

    # Calculate entropy for each connected component
    for component in nx.connected_components(G):
        subgraph = G.subgraph(component)
        component_constraints = list(nx.get_edge_attributes(subgraph, 'type').values())

        if component_constraints:
            count = Counter(component_constraints)
            total_constraints = len(component_constraints)

            if len(count) == 1:
                # Only one type of constraint, entropy is 0
                entropy = 0
            else:
                entropy = 0
                for constraint_type in all_constraint_types:
                    p = count[constraint_type] / total_constraints
                    if p > 0:
                        # Use log base 2 as an approximation
                        entropy -= p * math.log2(p) / math.log2(num_constraint_types)

            max_entropy = max(max_entropy, entropy)

    return max_entropy


def calculate_metrics(folder_path):
    """
    Calculate complexity metrics for each process model in a given folder and its subfolders.

    This function iterates over JSON files in a specified folder and its subfolders, calculates complexity metrics
    (size, density, separability, and constraint variability) for each process model, and stores the results
    in a pandas DataFrame.

    Args:
        folder_path (str): The path to the folder containing the JSON files.

    Returns:
        pd.DataFrame: A DataFrame containing the calculated metrics for each model.
    """
    metrics_results = []

    # Traverse the folder and subfolders
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    try:
                        content = f.read()
                        json_string = json.loads(content)
                        data = json.loads(json_string)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON from file {file_path}: {e}")
                        continue

                    # Calculate each metric
                    size_metric = calculate_size_metric(data)
                    density_metric = calculate_density_metric(data)
                    separability_metric = calculate_separability_metric(data)
                    constraint_variability_metric = calculate_constraint_variability_metric(data)

                    # Store the results for this model
                    metrics_results.append({
                        'File': file,
                        'Path': file_path,
                        'Size Metric': size_metric,
                        'Density Metric': density_metric,
                        'Separability Metric': separability_metric,
                        'Constraint Variability Metric': constraint_variability_metric
                    })

    # Convert results to a DataFrame for easy viewing
    df = pd.DataFrame(metrics_results)
    return df


def save_metrics_to_csv(metrics_df, folder_path, filename='complexity_metrics.csv'):
    """
    Save complexity metrics to a CSV file.

    This function takes a DataFrame containing complexity metrics, saves it to a specified folder as a CSV file,
    and ensures the file is named appropriately.

    Args:
        metrics_df (pd.DataFrame): The DataFrame containing the metrics to save.
        folder_path (str): The folder path where the CSV file will be saved.
        filename (str): The name of the CSV file (default: 'complexity_metrics.csv').

    Returns:
        str: The full path of the saved CSV file.
    """
    # Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)

    # Construct the full path for the CSV file
    file_path = os.path.join(folder_path, filename)

    # Save the DataFrame to CSV
    metrics_df.to_csv(file_path, index=False)

    return file_path


In [ ]:
df = calculate_metrics('/content/Nisaba evaluation')

In [ ]:
save_metrics_to_csv(df, '/content/')

In [ ]:
def decompose_model_components(decl):
    """
    Decompose a DECL model into its components: activities and constraints.
    """
    decl_model = copy.deepcopy(decl)
    decl_lines = decl_model.split("\n")
    decl_activities = [line[9:] for line in decl_lines if line.startswith("activity ")]

    # Process constraints with additional details
    decl_constraints = []

    # Unary templates and their semantics
    unary_templates = {
        "Absence": "Activation must not occur more than {n} times in the process.",
        "Exactly": "Activation must occur exactly {n} times in the process.",
        "Existence": "Activation must occur at least {n} times in the process.",
        "Init": "Activation must be the first activity to occur in the process.",
        "End": "Activation must be the last activity to occur in the process."
    }

    # Binary templates and their semantics
    binary_templates = {
        "Alternate Precedence": "If Target occurs, then Activation must have occurred before Target, and no Target occurs between Activation and Target.",
        "Alternate Response": "Each occurrence of Activation must be followed by Target and no Activation occurs between Activation and Target.",
        "Alternate Succession": "Each occurrence of Activation must be followed by Target, and each occurrence of Target must be preceded by Activation with no intermediate occurrences of Activation or Target.",
        "Chain Precedence": "Target can occur only if it is directly preceded by Activation.",
        "Chain Response": "Activation must be immediately followed by Target.",
        "Chain Succession": "Activation must be immediately followed by Target, and Target must be immediately preceded by Activation.",
        "Co-Existence": "If Activation occurs, then Target must also occur, and vice versa.",
        "Precedence": "Activation can occur only if it was preceded by Target.",
        "Responded Existence": "If Activation occurs, then Target must occur at some point in the process.",
        "Response": "If Activation occurs, then Target must eventually follow in the process.",
        "Succession": "Activation must be followed by Target, and Target must be preceded by Activation.",
        "Not Chain Succession": "Activation cannot be directly followed by Target, and Target cannot be directly preceded by Activation.",
        "Not Co-Existence": "Activation and Target cannot both occur in the process.",
        "Not Succession": "Activation and Target must not occur such that Activation is followed by Target or Target is preceded by Activation.",
        "Not Chain Precedence": "Target cannot directly follow Activation.",
        "Not Chain Response": "Activation cannot be directly followed by Target.",
        "Not Precedence": "Activation cannot be preceded by Target.",
        "Not Responded Existence": "If Activation occurs, then Target must not occur.",
        "Not Response": "Activation must not be followed by Target.",
        "Choice": "At least one of the activities Activation or Target must occur in the process.",
        "Exclusive Choice": "Exactly one of the activities Activation or Target must occur in the process."
    }

    for line in decl_lines:
        if "[" in line and "]" in line:
            constraint_info = {}
            # Full constraint
            constraint_info["mpdeclare"] = line

            # Extracting template, activation, and target
            template = line.split("[")[0].strip()
            activation_target = line.split("[")[1].split("]")[0]

            if "," in activation_target:
                constraint_type = "binary"

                # Check if template matches any binary templates
                if template in binary_templates:
                    order = "normal"

                    if template in ["Alternate Precedence", "Chain Precedence", "Precedence",
                                    "Not Chain Precedence", "Not Precedence"]:
                        order = "reverse"

                    if order == "normal":
                        activation, target = map(str.strip, activation_target.split(","))
                    elif order == "reverse":
                        target, activation = map(str.strip, activation_target.split(","))

                else:
                    # Default to "normal" order if template is unknown
                    activation, target = map(str.strip, activation_target.split(","))

                # Assign semantics for binary templates
                semantics = binary_templates.get(template, "").replace("Activation", activation).replace("Target", target)

            else:
                constraint_type = "unary"
                activation = activation_target.strip()
                target = None

                # Extract cardinality if the template is one of the unary templates
                cardinality = 1
                if template in unary_templates:
                    match = re.search(r'\d+', template)
                    if match:
                        cardinality = int(match.group())
                    template = re.sub(r'\d+', '', template)  # Remove the number from the template

                # Assign semantics for unary templates
                if template in unary_templates:
                    semantics = unary_templates[template].replace("{n}", str(cardinality)).replace("Activation", activation)
                else:
                    semantics = ""

            constraint_info["template"] = template
            constraint_info["type"] = constraint_type
            constraint_info["activation"] = activation
            constraint_info["target"] = target
            constraint_info["semantics"] = semantics

            # Add cardinality if it's a unary constraint with one of the relevant templates
            if constraint_type == "unary" and template in unary_templates:
                constraint_info["cardinality"] = cardinality

            decl_constraints.append(constraint_info)

    return {
        "activities": decl_activities,
        "constraints": decl_constraints,
    }

def semantic_pairing(list_A, list_B, embedding_model):
    """
    Perform semantic pairing between two lists of activities based on their embeddings.
    """
    # Encode activities from both lists
    embedding_A = embedding_model.encode(list_A)
    embedding_B = embedding_model.encode(list_B)

    # Normalize the embedding vectors along the rows
    embedding_A = (embedding_A.T / np.linalg.norm(embedding_A, axis=1)).T
    embedding_B = (embedding_B.T / np.linalg.norm(embedding_B, axis=1)).T

    # Compute cost matrix as 1 minus the dot product of unit vectors
    cost_matrix = 1 - embedding_A @ embedding_B.T

    # Solve the assignment problem using the Jonker-Volgenant algorithm
    row_ind, col_ind, cost = lapjv(cost_matrix)

    # Extract assignment costs
    assignment_cost = []
    for i, j in zip(np.arange(cost_matrix.shape[0]), row_ind):
        assignment_cost.append(cost_matrix[i, j])

    # Return the lowest-cost bijection pairing between activities in A and B
    return list(zip(list_A, np.array(list_B)[row_ind], assignment_cost))


def compute_semantic_similarity(decl_original, decl_reconstructed, embedding_model):
    """
    Computes the semantic and constraint-based distance between two declarative models.

    Args:
        decl_original (dict): Original declarative model components.
        decl_reconstructed (dict): Reconstructed declarative model components.
        embedding_model: Model used for semantic pairing of activities.

    Returns:
        float: Distance between the two models based on constraints and semantic activity substitution.
    """
    # Step 1: Decompose the models
    decl_original = decompose_model_components(decl_original)
    decl_reconstructed = decompose_model_components(decl_reconstructed)

    # Extract constraints
    decl_original_constraints = [
        c["mpdeclare"].split("]")[0] + "]" for c in decl_original["constraints"]
    ]
    decl_reconstructed_constraints = [
        c["mpdeclare"].split("]")[0] + "]" for c in decl_reconstructed["constraints"]
    ]

    # Step 2: Pair activities semantically

    # Ensure that semantics lists have the same length by padding the shorter one
    if len(decl_original["activities"]) < len(decl_reconstructed["activities"]):
        decl_original["activities"] += [''] * (
            len(decl_reconstructed["activities"]) - len(decl_original["activities"])
        )
    elif len(decl_reconstructed["activities"]) < len(decl_original["activities"]):
        decl_reconstructed["activities"] += [''] * (
            len(decl_original["activities"]) - len(decl_reconstructed["activities"])
        )

    # Perform semantic pairing of activities
    activities_pairing = semantic_pairing(
        decl_original["activities"], decl_reconstructed["activities"], embedding_model
    )

    # Create a mapping from original activities to substituted activities
    activity_substitution = {
        act1: activity_sub
        for activity_sub, (act1, act2, _) in zip("ABCDEFGHIJKLMNOPQ", activities_pairing)
    }
    activity_substitution.update({
        act2: activity_sub
        for activity_sub, (act1, act2, _) in zip("ABCDEFGHIJKLMNOPQ", activities_pairing)
    })

    # Replace activities in constraints with substituted activities
    decl_original_constraints = [
        replace_activities_in_constraint(c, activity_substitution)
        for c in decl_original_constraints
    ]
    decl_reconstructed_constraints = [
        replace_activities_in_constraint(c, activity_substitution)
        for c in decl_reconstructed_constraints
    ]

    # Step 3: Compute distance between the sets of constraints
    decl1_set = set(decl_original_constraints)
    decl2_set = set(decl_reconstructed_constraints)
    distance = 1 - (len(decl1_set.intersection(decl2_set)) / len(decl1_set.union(decl2_set)))

    return distance


def replace_activities_in_constraint(constraint, activity_substitution):
    """
    Replaces activities in a constraint with their substituted values.

    Args:
        constraint (str): The constraint in which to replace activities.
        activity_substitution (dict): Mapping of original activities to substituted activities.

    Returns:
        str: Updated constraint with substituted activities.
    """
    for act, sub in activity_substitution.items():
        constraint = constraint.replace(act, sub)
    return constraint

####Readability

In [ ]:
# Initialize spaCy model with the readability component
nlp = spacy.load('en_core_web_sm')

# Add readability component to the pipeline
if 'readability' not in nlp.pipe_names:
    nlp.add_pipe(Readability(), last=True)

# Function to convert markdown to text
def markdown_to_text(markdown_string):
    # Convert markdown to HTML
    html = markdown(markdown_string)
    # Use BeautifulSoup to extract text from HTML
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()

# Function to calculate the Flesch Reading Ease score
def calculate_readability(doc):
    return {
        'filename': '',  # Placeholder, will be set later
        'flesch_reading_ease': doc._.flesch_kincaid_reading_ease
    }

# Function to process a folder
def process_folder(folder_path, output_csv):
    results = []

    # Walk through the files in the folder
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):  # Look for .txt files
                filepath = os.path.join(root, file)
                with open(filepath, 'r', encoding='utf-8') as f:
                    markdown_content = f.read()
                    # Convert markdown to plain text
                    text = markdown_to_text(markdown_content)
                    # Process text with spaCy
                    doc = nlp(text)
                    # Calculate the Flesch Reading Ease score
                    scores = calculate_readability(doc)
                    # Set the filename in the scores dictionary
                    scores['filename'] = file
                    # Append results
                    results.append(scores)

    # Save results to CSV
    df = pd.DataFrame(results)
    # Ensure the DataFrame has only the desired columns in order
    df = df[['filename', 'flesch_reading_ease']]
    # Sort the DataFrame alphabetically by filename
    df = df.sort_values('filename')
    # Save the DataFrame to CSV
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# Specify the paths of descriptions and output
folder_path = '/content/Descriptions'  # Path to the folder containing markdown files
output_csv = 'readability_scores.csv'

# Process the folder
process_folder(folder_path, output_csv)


#Application - Deprecated

In [ ]:
def handle_input(input_type, text_input, file_input):
    declare_model = ExtendedDeclareModel()
    if input_type == "Lines":
        model = text_input
    elif input_type == "File" and file_input is not None:
        try:
            with open(file_input.name, 'r', encoding='utf-8') as f:
                model = f.read()
        except Exception as e:
            return f"Error reading file: {str(e)}"
    else:
        return "Please provide valid input."

    try:
        declare_model.parse_from_string(model)
        declare_model.declare_model_lines
        return "Model parsed successfully."
    except Exception as e:
        return f"Failed to parse model: {str(e)}"

def show_additional_outputs(output):
    if "Model parsed successfully." in output:
        return gr.update(visible=True), gr.update(visible=True)
    else:
        return gr.update(visible=False), gr.update(visible=False)

with gr.Blocks() as demo:
    gr.Markdown("# MP-Declare Model Description Generator")

    with gr.Row():
        input_type = gr.Radio(choices=["Lines", "File"], label="Choose Import Type")

    text_input = gr.Textbox(label="Input Model Lines", visible=False)
    file_input = gr.File(label="Upload .decl File", visible=False)

    def update_ui(choice):
        if choice == "Lines":
            return gr.update(visible=True), gr.update(visible=False)
        elif choice == "File":
            return gr.update(visible=False), gr.update(visible=True)

    input_type.change(update_ui, input_type, [text_input, file_input])

    submit_btn = gr.Button("Submit")

    # Make the output textbox non-editable by setting interactive=False
    output = gr.Textbox(label="Output", interactive=False)

    # New textboxes for intermediary and continuous descriptions, initially hidden
    intermediary_descriptions = gr.Textbox(label="Intermediary Descriptions", visible=False)
    continuous_descriptions = gr.Textbox(label="Continuous Descriptions", visible=False)

    # First, handle input and display the output
    submit_btn.click(handle_input, [input_type, text_input, file_input], output)

    # Then, based on the output, control the visibility of the additional textboxes
    output.change(show_additional_outputs, output, [intermediary_descriptions, continuous_descriptions])

demo.launch()


In [ ]:
def format_prompt(declare_model_lines):
    model = "\n".join(declare_model_lines)
    prompt = "New model to generate description of activities, attributes, binds, and constraints based on the template: \n" + model
    return prompt

def process_intermediate_descriptions(tool_calls):
    model = {"Activities": [], "Attributes": [], "Binds": [], "Constraints": []}
    try:
        for item in tool_calls:
            func_args = json.loads(item.function.arguments)
            for key, value in func_args.items():
                for v in value:
                    model[key.capitalize()].append(f"{v['name']}: {v['description']}")
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error processing descriptions: {e}")
        return ""

    model_string = ""
    for key, descriptions in model.items():
        model_string += f"{key}:\n"
        for desc in descriptions:
            model_string += f"  - {desc}\n"

    return model_string

def generate_intermediary_description(declare_model_lines, decltotextassistant, toolsdescription):
    model_prompt = format_prompt(declare_model_lines)
    messagesmptodesc = [
        {"role": "system", "content": decltotextassistant},
        {"role": "user", "content": model_prompt}
    ]

    response = chat_completion_request(
        messages=messagesmptodesc,
        tools=toolsdescription,
        tool_choice="auto",
        temperature=0.2
    )

    if response:
        description_string = process_intermediate_descriptions(response.choices[0].message.tool_calls)
        return description_string
    else:
        return "Failed to generate description."

def handle_input(input_type, text_input, file_input, decltotextassistant, toolsdescription):
    declare_model = ExtendedDeclareModel()
    if input_type == "Lines":
        model = text_input
    elif input_type == "File" and file_input is not None:
        try:
            with open(file_input.name, 'r', encoding='utf-8') as f:
                model = f.read()
        except Exception as e:
            return f"Error reading file: {str(e)}", "", ""
    else:
        return "Please provide valid input.", "", ""

    try:
        declare_model.parse_from_string(model)
        declare_model_lines = declare_model.declare_model_lines

        intermediary_description = generate_intermediary_description(declare_model_lines, decltotextassistant, toolsdescription)

        return "Model parsed successfully.", intermediary_description, ""
    except Exception as e:
        return f"Failed to parse model: {str(e)}", "", ""

def show_additional_outputs(output):
    if "Model parsed successfully." in output:
        return gr.update(visible=True), gr.update(visible=True)  # Show both textboxes
    else:
        return gr.update(visible=False), gr.update(visible=False)

# Wrapper function that includes the external arguments
def handle_input_wrapper(input_type, text_input, file_input):
    return handle_input(input_type, text_input, file_input, decltotextassistant, toolsdescription)

with gr.Blocks() as demo:
    gr.Markdown("# MP-Declare Model Description Generator")

    with gr.Row():
        input_type = gr.Radio(choices=["Lines", "File"], label="Choose Import Type")
    text_input = gr.Textbox(label="Input Model Lines", visible=False)
    file_input = gr.File(label="Upload .decl File", visible=False)

    def update_ui(choice):
        if choice == "Lines":
            return gr.update(visible=True), gr.update(visible=False)
        elif choice == "File":
            return gr.update(visible=False), gr.update(visible=True)

    input_type.change(update_ui, input_type, [text_input, file_input])

    submit_btn = gr.Button("Submit")

    output = gr.Textbox(label="Output", interactive=False, lines=2)
    intermediary_descriptions = gr.Textbox(
        label="Intermediary Descriptions",
        visible=False,
        interactive=True,  # Make it adjustable
        lines=10  # Set initial height
    )
    final_descriptions = gr.Textbox(
        label="Final Descriptions",
        visible=False,
        interactive=True,  # Make it adjustable
        lines=10  # Set initial height
    )

    # Handle input and display the output and intermediary descriptions using the wrapper function
    submit_btn.click(fn=handle_input_wrapper, inputs=[input_type, text_input, file_input], outputs=[output, intermediary_descriptions])

    # Control visibility of both intermediary and final descriptions
    output.change(show_additional_outputs, output, [intermediary_descriptions, final_descriptions])

demo.launch()


In [ ]:
def generate_final_descriptions(description_string, enhdescassistant):
    messagesendesc = [
        {"role": "system", "content": enhdescassistant},
        {"role": "user", "content": "New model to generate description: " + description_string}
    ]

    responseendesc = chat_completion_request(
        messages=messagesendesc,
        temperature=0.2
    )

    if responseendesc:
        final_description = responseendesc.choices[0].message.content
        return final_description
    else:
        return "Failed to generate enhanced description."

def handle_input(input_type, text_input, file_input, decltotextassistant, toolsdescription, enhdescassistant):
    declare_model = ExtendedDeclareModel()
    if input_type == "Lines":
        model = text_input
    elif input_type == "File" and file_input is not None:
        try:
            with open(file_input.name, 'r', encoding='utf-8') as f:
                model = f.read()
        except Exception as e:
            return f"Error reading file: {str(e)}", "", ""
    else:
        return "Please provide valid input.", "", ""

    try:
        declare_model.parse_from_string(model)
        declare_model_lines = declare_model.declare_model_lines

        intermediary_description = generate_intermediary_description(declare_model_lines, decltotextassistant, toolsdescription)
        final_description = generate_final_descriptions(intermediary_description, enhdescassistant)

        return "Model parsed successfully.", intermediary_description, final_description
    except Exception as e:
        return f"Failed to parse model: {str(e)}", "", ""

def show_additional_outputs(output):
    if "Model parsed successfully." in output:
        return gr.update(visible=True), gr.update(visible=True)  # Show both textboxes
    else:
        return gr.update(visible=False), gr.update(visible=False)

# Wrapper function that includes the external arguments
def handle_input_wrapper(input_type, text_input, file_input):
    return handle_input(input_type, text_input, file_input, decltotextassistant, toolsdescription, enhdescassistant)

with gr.Blocks() as demo:
    gr.Markdown("# MP-Declare Model Description Generator")

    with gr.Row():
        input_type = gr.Radio(choices=["Lines", "File"], label="Choose Import Type")
    text_input = gr.Textbox(label="Input Model Lines", visible=False)
    file_input = gr.File(label="Upload .decl File", visible=False)

    def update_ui(choice):
        if choice == "Lines":
            return gr.update(visible=True), gr.update(visible=False)
        elif choice == "File":
            return gr.update(visible=False), gr.update(visible=True)

    input_type.change(update_ui, input_type, [text_input, file_input])

    submit_btn = gr.Button("Submit")

    output = gr.Textbox(label="Output", interactive=False, lines=1)
    intermediary_descriptions = gr.Textbox(
        label="Intermediary Descriptions",
        visible=False,
        interactive=False,  # Make it adjustable
        lines=10  # Set initial height
    )
    final_descriptions = gr.Textbox(
        label="Final Descriptions",
        visible=False,  # This ensures it only shows after submit
        interactive=False,  # Still allow editing if necessary
        lines=10  # Set initial height
    )

    # Handle input and display the output and intermediary descriptions using the wrapper function
    submit_btn.click(fn=handle_input_wrapper, inputs=[input_type, text_input, file_input], outputs=[output, intermediary_descriptions, final_descriptions])

    # Control visibility of both intermediary and final descriptions
    output.change(show_additional_outputs, output, [intermediary_descriptions, final_descriptions])

demo.launch()
